In [1]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt

%load_ext nb_black

d:\py\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
d:\py\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


<IPython.core.display.Javascript object>

In [2]:
excel_file = r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\data.xlsx"

<IPython.core.display.Javascript object>

In [3]:
lifetime = 25  # 使用年限

r = 0.075  # 贴现率
rv = 0.05  # 资产残值率
capacity_factor_ammonia = 8000 / 8760

a = -1
for i in range(lifetime + 1):
    a = a + 1 / ((1 + r) ** i)
a

11.146945860662251

<IPython.core.display.Javascript object>

In [4]:
def Cal_LCO(
    C_initial,
    C_variable_year,
    C_electricity_year,
    C_fuel_year,
    C_energy_feedstock_year,
    C_water_year,
    C_catalyst_year,
    C_chemical_feedstock_year,
    C_fixed_operation_year,
    Q_year,
    Vr,
    lifetime=lifetime,
    r=r,
):

    # levelized cost of ammonia
    LCO = (C_initial + C_variable_year * a - Vr / (1 + r) ** lifetime) / (
        Q_year * 1000 * a
    )

    CAPEX = (C_initial - Vr / (1 + r) ** lifetime) / (Q_year * 1000 * a)
    LCOA_electricity = C_electricity_year
    LCOA_fuel = C_fuel_year
    LCOA_energy_feedstock = C_energy_feedstock_year
    OPEX = (
        C_variable_year / (Q_year * 1000)
        - LCOA_electricity
        - LCOA_fuel
        - LCOA_energy_feedstock
    )

    return (LCO, CAPEX, LCOA_electricity, LCOA_fuel, LCOA_energy_feedstock, OPEX)

<IPython.core.display.Javascript object>

## Province

In [ ]:
Province_data = pd.read_excel(excel_file, sheet_name="Province")

Province = [
    "Anhui",
    "Beijing",
    "Chongqing",
    "Fujian",
    "Gansu",
    "Guangdong",
    "Guangxi",
    "Guizhou",
    "Hainan",
    "Hebei",
    "Heilongjiang",
    "Henan",
    "Hubei",
    "Hunan",
    "InnerMongolia",
    "Jiangsu",
    "Jiangxi",
    "Jilin",
    "Liaoning",
    "Ningxia",
    "Qinghai",
    "Shaanxi",
    "Shandong",
    "Shanghai",
    "Shanxi",
    "Sichuan",
    "Tianjin",
    "Xinjiang",
    "Yunnan",
    "Zhejiang",
    "National",
]
Year = ["2018", "2020", "2025", "2030", "2035", "2040", "2045", "2050", "2055", "2060"]
Year1 = [
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
    "2024",
    "2025",
    "2026",
    "2027",
    "2028",
    "2029",
    "2030",
    "2031",
    "2032",
    "2033",
    "2034",
    "2035",
    "2036",
    "2037",
    "2038",
    "2039",
    "2040",
    "2041",
    "2042",
    "2043",
    "2044",
    "2045",
    "2046",
    "2047",
    "2048",
    "2049",
    "2050",
    "2051",
    "2052",
    "2053",
    "2054",
    "2055",
    "2056",
    "2057",
    "2058",
    "2059",
    "2060",
]

## CG

In [6]:
CG_data = pd.read_excel(excel_file, sheet_name="CG")
Q_CG_year = 500000 * capacity_factor_ammonia  # CG每年产氨量

# 固定成本
CG_initial = (
    CG_data.iloc[16:26, 2].sum(axis=0)
    / (200000 / (Q_CG_year / capacity_factor_ammonia)) ** 0.7
)
Vr_CG = CG_initial * rv  # 固定资产残值

# 运营成本
CG_fixed_operation = 0.042 * CG_initial  # 假定年维护费用为初始投资的4.2%

# CG的可变运营成本
CG_water = CG_data.iloc[50, 2] * CG_data.iloc[50, 4]
CG_catalyst = np.dot(CG_data.iloc[56, 2], CG_data.iloc[56, 4])
CG_chemical_feedstock = np.dot(CG_data.iloc[51:58, 2], CG_data.iloc[51:58, 4])

# 省级和时间维度，省级维度最后一行为全国层面
CG_electricity_province = [0] * 31
CG_fuel_province = [0] * 31
CG_energy_feedstock_province = [0] * 31
CG_variable_operation_province = [0] * 31

for j in range(31):
    CG_energy_feedstock_province[j] = Province_data.iloc[j + 2, 4] * CG_data.iloc[47, 4]
    CG_fuel_province[j] = Province_data.iloc[j + 2, 3] * CG_data.iloc[48, 4]
    CG_electricity_province[j] = Province_data.iloc[j + 2, 2] * CG_data.iloc[49, 4]
    CG_variable_operation_province[j] = (
        CG_electricity_province[j]
        + CG_fuel_province[j]
        + CG_energy_feedstock_province[j]
        + CG_water
        + CG_catalyst
        + CG_chemical_feedstock
    )

# CG每年每kg合成氨的变动资本总和
CG_variable_province = [0] * 31
for j in range(31):
    CG_variable_province[j] = (
        CG_fixed_operation + CG_variable_operation_province[j] * Q_CG_year * 1000
    )

# CG各期各省每kg的LCOA
LCOA_CG_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_CG_results_province_future[j, i] = (
            CG_initial + CG_variable_province[j] * a - Vr_CG / ((1 + r) ** lifetime)
        ) / (Q_CG_year * 1000 * a)

LCOA_CG_results_province_future_df = pd.DataFrame(
    LCOA_CG_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# CG 2018年和2060年全国LCOA的成本结构
LCOA_CG_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_initial,
        C_variable_year=CG_variable_province[30],
        C_electricity_year=CG_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock,
        C_fixed_operation_year=CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_CG_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_initial,
        C_variable_year=CG_variable_province[30],
        C_electricity_year=CG_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock,
        C_fixed_operation_year=CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2060"],
)
LCOA_CG_results_province_future_df

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,3.724672,3.724672,3.724672,3.724672,3.724672,3.724672,3.724672,3.724672,3.724672,3.724672
Beijing,3.577310,3.577310,3.577310,3.577310,3.577310,3.577310,3.577310,3.577310,3.577310,3.577310
Chongqing,3.692834,3.692834,3.692834,3.692834,3.692834,3.692834,3.692834,3.692834,3.692834,3.692834
Fujian,3.709195,3.709195,3.709195,3.709195,3.709195,3.709195,3.709195,3.709195,3.709195,3.709195
Gansu,3.442416,3.442416,3.442416,3.442416,3.442416,3.442416,3.442416,3.442416,3.442416,3.442416
Guangdong,3.749437,3.749437,3.749437,3.749437,3.749437,3.749437,3.749437,3.749437,3.749437,3.749437
Guangxi,3.945035,3.945035,3.945035,3.945035,3.945035,3.945035,3.945035,3.945035,3.945035,3.945035
Guizhou,3.464748,3.464748,3.464748,3.464748,3.464748,3.464748,3.464748,3.464748,3.464748,3.464748
Hainan,3.663596,3.663596,3.663596,3.663596,3.663596,3.663596,3.663596,3.663596,3.663596,3.663596
Hebei,3.471977,3.471977,3.471977,3.471977,3.471977,3.471977,3.471977,3.471977,3.471977,3.471977


<IPython.core.display.Javascript object>

## CG+CCS

In [7]:
CG_CCS_data = pd.read_excel(excel_file, sheet_name="CG_CCS")

# CG+CCS固定资产投资=CG+CCS
Future_data = pd.read_excel(excel_file, sheet_name="Cost")

CG_capture_initial_future = Future_data.iloc[17, 4:14] * 3.70816230293156 * Q_CG_year
CG_storage_initial_future = Future_data.iloc[23, 4:14] * 3.70816230293156 * Q_CG_year
CG_transport_initial_future = Future_data.iloc[20, 4:14] * 3.70816230293156 * Q_CG_year

Carbon_transport_distance_province = [0] * 31
for j in range(31):
    Carbon_transport_distance_province[j] = Province_data.iloc[j + 2, 8]

CG_transport_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_transport_initial_province_future[j, i] = (
            CG_transport_initial_future[i] * Carbon_transport_distance_province[j]
        )

CG_CCS_initial_province_future = np.zeros((31, 10))
Vr_CG_CCS_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_CCS_initial_province_future[j, i] = (
            CG_initial
            + CG_capture_initial_future[i]
            + CG_storage_initial_future[i]
            + CG_transport_initial_province_future[j, i]
        )
        Vr_CG_CCS_province_future[j, i] = CG_CCS_initial_province_future[j, i] * rv

# CG+CCS运营成本
# CG+CCS固定运营成本
CG_capture_fixed_operation_future = [0] * 10
CG_storage_fixed_operation_future = [0] * 10
for i in range(10):
    CG_capture_fixed_operation_future[i] = 0.0367 * CG_capture_initial_future[i]
    CG_storage_fixed_operation_future[i] = 0.04 * CG_storage_initial_future[i]

CG_transport_fixed_operation_province_future = np.zeros((31, 10))
CG_CCS_fixed_operation_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_transport_fixed_operation_province_future[j, i] = (
            0.0222 * CG_transport_initial_province_future[j, i]
        )
        CG_CCS_fixed_operation_province_future[j, i] = (
            CG_capture_fixed_operation_future[i]
            + CG_storage_fixed_operation_future[i]
            + CG_transport_fixed_operation_province_future[j, i]
        )

# CG+CCS可变运营成本
CG_CCS_water = CG_CCS_data.iloc[69, 2] * CG_CCS_data.iloc[69, 4]
CG_CCS_chemical_feedstock = np.dot(
    CG_CCS_data.iloc[70:73, 2], CG_CCS_data.iloc[70:73, 4]
)

CG_CCS_electricity_province = [0] * 31
CG_CCS_variable_operation_province = [0] * 31
for j in range(31):
    CG_CCS_electricity_province[j] = (
        Province_data.iloc[j + 2, 2] * CG_CCS_data.iloc[73, 4]
    )
    CG_CCS_variable_operation_province[j] = (
        CG_CCS_electricity_province[j] + CG_CCS_water + CG_CCS_chemical_feedstock
    )

# CG+CCS每年每kg合成氨的变动资本总和
CG_CCS_variable_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_CCS_variable_province_future[j, i] = (
            CG_CCS_fixed_operation_province_future[j, i]
            + CG_fixed_operation
            + (
                CG_CCS_variable_operation_province[j]
                + CG_variable_operation_province[j]
            )
            * Q_CG_year
            * 1000
        )


# CG+CCS各期各省每kg的LCOA
LCOA_CG_CCS_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_CG_CCS_results_province_future[j, i] = (
            CG_CCS_initial_province_future[j, i]
            + CG_CCS_variable_province_future[j, i] * a
            - Vr_CG_CCS_province_future[j, i] / (1 + r) ** lifetime
        ) / (Q_CG_year * 1000 * a)

LCOA_CG_CCS_results_province_future_df = pd.DataFrame(
    LCOA_CG_CCS_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# CG+CCS 2018年和2060年全国LCOA的成本结构
LCOA_CG_CCS_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_CCS_initial_province_future[30, 0],
        C_variable_year=CG_CCS_variable_province_future[30, 0],
        C_electricity_year=CG_electricity_province[30]
        + CG_CCS_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water + CG_CCS_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock + CG_CCS_chemical_feedstock,
        C_fixed_operation_year=CG_CCS_fixed_operation_province_future[30, 0]
        + CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG_CCS_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_CG_CCS_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_CCS_initial_province_future[30, 9],
        C_variable_year=CG_CCS_variable_province_future[30, 9],
        C_electricity_year=CG_electricity_province[30]
        + CG_CCS_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water + CG_CCS_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock + CG_CCS_chemical_feedstock,
        C_fixed_operation_year=CG_CCS_fixed_operation_province_future[30, 9]
        + CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG_CCS_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)
LCOA_CG_CCS_results_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3718104960.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  CG_transport_initial_future[i] * Carbon_transport_distance_province[j]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3718104960.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + CG_capture_initial_future[i]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3718104960.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.il

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,5.161054,5.086865,4.901390,4.715915,4.521170,4.409858,4.358862,4.307867,4.266125,4.224383
Beijing,5.013872,4.939683,4.754208,4.568733,4.373988,4.262676,4.211680,4.160685,4.118943,4.077201
Chongqing,5.129026,5.054836,4.869362,4.683887,4.489142,4.377830,4.326834,4.275839,4.234097,4.192355
Fujian,6.987851,6.854223,6.520155,6.186086,5.842747,5.582842,5.513272,5.443702,5.346237,5.248773
Gansu,4.877989,4.803799,4.618324,4.432849,4.238104,4.126792,4.075796,4.024801,3.983059,3.941317
Guangdong,7.028205,6.894577,6.560509,6.226440,5.883101,5.623196,5.553626,5.484056,5.386591,5.289127
Guangxi,7.223889,7.090262,6.756193,6.422124,6.078785,5.818880,5.749310,5.679740,5.582275,5.484811
Guizhou,4.900653,4.826463,4.640988,4.455514,4.260768,4.149457,4.098461,4.047465,4.005724,3.963982
Hainan,5.100027,5.025837,4.840362,4.654888,4.460142,4.348831,4.297835,4.246839,4.205098,4.163356
Hebei,4.907938,4.833748,4.648273,4.462798,4.268053,4.156741,4.105746,4.054750,4.013008,3.971266


<IPython.core.display.Javascript object>

## SMR

In [8]:
SMR_data = pd.read_excel(excel_file, sheet_name="SMR")
Q_SMR_year = 500000 * capacity_factor_ammonia  # 天然气制氨年产量为50万吨

# 固定成本
SMR_initial = SMR_data.iloc[16:27, 2].sum(axis=0)  # 固定资本
Vr_SMR = SMR_initial * rv  # 固定资产残值

# 运营成本
# SMR的年固定变动资本
SMR_fixed_operation = 0.0282 * SMR_initial

# SMR的可变运营成本
SMR_water = SMR_data.iloc[52, 2] * SMR_data.iloc[52, 4]
SMR_catalyst = np.dot(SMR_data.iloc[53:60, 2], SMR_data.iloc[53:60, 4])
SMR_chemical_feedstock = 0

SMR_electricity_province = [0] * 31
SMR_fuel_province = [0] * 31
SMR_energy_feedstock_province = [0] * 31
SMR_variable_operation_province = [0] * 31

for j in range(31):
    SMR_fuel_province[j] = Province_data.iloc[j + 2, 5] * SMR_data.iloc[50, 4]
    SMR_energy_feedstock_province[j] = (
        Province_data.iloc[j + 2, 5] * SMR_data.iloc[49, 4]
    )

    SMR_electricity_province[j] = Province_data.iloc[j + 2, 2] * SMR_data.iloc[51, 4]
    SMR_variable_operation_province[j] = (
        SMR_fuel_province[j]
        + SMR_energy_feedstock_province[j]
        + SMR_electricity_province[j]
        + SMR_water
        + SMR_catalyst
        + SMR_chemical_feedstock
    )

# SMR每年每kg合成氨的变动资本总和
SMR_variable_province = [0] * 31
for j in range(31):
    SMR_variable_province[j] = (SMR_fixed_operation) + (
        (SMR_variable_operation_province[j]) * Q_SMR_year * 1000
    )

# SMR各期各省每kg的LCOA
LCOA_SMR_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_SMR_results_province_future[j, i] = (
            SMR_initial + SMR_variable_province[j] * a - Vr_SMR / ((1 + r) ** lifetime)
        ) / (Q_SMR_year * 1000 * a)

LCOA_SMR_results_province_future_df = pd.DataFrame(
    LCOA_SMR_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# SMR 2018年和2060年全国LCOA的成本结构
LCOA_SMR_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_initial,
        C_variable_year=SMR_variable_province[30],
        C_electricity_year=SMR_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock,
        C_fixed_operation_year=SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2018"],
)

LCOA_SMR_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_initial,
        C_variable_year=SMR_variable_province[30],
        C_electricity_year=SMR_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock,
        C_fixed_operation_year=SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_SMR_results_province_future_df

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,4.327076,4.327076,4.327076,4.327076,4.327076,4.327076,4.327076,4.327076,4.327076,4.327076
Beijing,4.135376,4.135376,4.135376,4.135376,4.135376,4.135376,4.135376,4.135376,4.135376,4.135376
Chongqing,3.376092,3.376092,3.376092,3.376092,3.376092,3.376092,3.376092,3.376092,3.376092,3.376092
Fujian,4.616607,4.616607,4.616607,4.616607,4.616607,4.616607,4.616607,4.616607,4.616607,4.616607
Gansu,3.501823,3.501823,3.501823,3.501823,3.501823,3.501823,3.501823,3.501823,3.501823,3.501823
Guangdong,5.013750,5.013750,5.013750,5.013750,5.013750,5.013750,5.013750,5.013750,5.013750,5.013750
Guangxi,4.830713,4.830713,4.830713,4.830713,4.830713,4.830713,4.830713,4.830713,4.830713,4.830713
Guizhou,4.386944,4.386944,4.386944,4.386944,4.386944,4.386944,4.386944,4.386944,4.386944,4.386944
Hainan,4.464924,4.464924,4.464924,4.464924,4.464924,4.464924,4.464924,4.464924,4.464924,4.464924
Hebei,4.149443,4.149443,4.149443,4.149443,4.149443,4.149443,4.149443,4.149443,4.149443,4.149443


<IPython.core.display.Javascript object>

## SMR+CCS

In [9]:
SMR_CCS_data = pd.read_excel(excel_file, sheet_name="SMR_CCS")

# SMR+CCS固定资产投资=SMR+CCS
Future_data = pd.read_excel(excel_file, sheet_name="Cost")

SMR_capture_initial_future = Future_data.iloc[17, 4:14] * 1.3585 * Q_SMR_year
SMR_storage_initial_future = Future_data.iloc[23, 4:14] * 1.3585 * Q_SMR_year
SMR_transport_initial_future = Future_data.iloc[20, 4:14] * 1.3585 * Q_SMR_year

Carbon_transport_distance_province = [0] * 31
for j in range(31):
    Carbon_transport_distance_province[j] = Province_data.iloc[j + 2, 8]

SMR_transport_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_transport_initial_province_future[j, i] = (
            SMR_transport_initial_future[i] * Carbon_transport_distance_province[j]
        )

SMR_CCS_initial_province_future = np.zeros((31, 10))
Vr_SMR_CCS_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_CCS_initial_province_future[j, i] = (
            SMR_initial
            + SMR_capture_initial_future[i]
            + SMR_storage_initial_future[i]
            + SMR_transport_initial_province_future[j, i]
        )
        Vr_SMR_CCS_province_future[j, i] = SMR_CCS_initial_province_future[j, i] * rv

# SMR+CCS运营成本
# SMR+CCS固定运营成本
SMR_capture_fixed_operation_future = [0] * 10
SMR_storage_fixed_operation_future = [0] * 10
for j in range(10):
    SMR_capture_fixed_operation_future[i] = 0.0367 * SMR_capture_initial_future[i]
    SMR_storage_fixed_operation_future[i] = 0.04 * SMR_storage_initial_future[i]

SMR_transport_fixed_operation_province_future = np.zeros((31, 10))
SMR_CCS_fixed_operation_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_transport_fixed_operation_province_future[j, i] = (
            0.0222 * SMR_transport_initial_province_future[j, i]
        )
        SMR_CCS_fixed_operation_province_future[j, i] = (
            SMR_capture_fixed_operation_future[i]
            + SMR_storage_fixed_operation_future[i]
            + SMR_transport_fixed_operation_province_future[j, i]
        )

# SMR+CCS可变运营成本
SMR_CCS_water = SMR_CCS_data.iloc[76, 2] * SMR_CCS_data.iloc[76, 4]
SMR_CCS_chemical_feedstock = np.dot(
    SMR_CCS_data.iloc[77:80, 2], SMR_CCS_data.iloc[77:80, 4]
)

SMR_CCS_electricity_province = [0] * 31
SMR_CCS_variable_operation_province = [0] * 31
for j in range(31):
    SMR_CCS_electricity_province[j] = (
        Province_data.iloc[j + 2, 2] * SMR_CCS_data.iloc[80, 4]
    )
    SMR_CCS_variable_operation_province[j] = (
        SMR_CCS_electricity_province[j] + SMR_CCS_water + SMR_CCS_chemical_feedstock
    )

# SMR+CCS每年每kg合成氨的变动资本总和
SMR_CCS_variable_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_CCS_variable_province_future[j, i] = (
            SMR_CCS_fixed_operation_province_future[j, i] + SMR_fixed_operation
        ) + (
            (
                SMR_CCS_variable_operation_province[j]
                + SMR_variable_operation_province[j]
            )
            * Q_SMR_year
            * 1000
        )

# SMR+CCS各期各省每kg的LCOA
LCOA_SMR_CCS_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_SMR_CCS_results_province_future[j, i] = (
            SMR_CCS_initial_province_future[j, i]
            + SMR_CCS_variable_province_future[j, i] * a
            - Vr_SMR_CCS_province_future[j, i] / (1 + r) ** lifetime
        ) / (Q_SMR_year * 1000 * a)

LCOA_SMR_CCS_results_province_future_df = pd.DataFrame(
    LCOA_SMR_CCS_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# SMR+CCS 2018年和2060年全国LCOA的成本结构
LCOA_SMR_CCS_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_CCS_initial_province_future[30, 0],
        C_variable_year=SMR_CCS_variable_province_future[30, 0],
        C_electricity_year=SMR_electricity_province[30]
        + SMR_CCS_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water + SMR_CCS_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock + SMR_CCS_chemical_feedstock,
        C_fixed_operation_year=SMR_CCS_fixed_operation_province_future[30, 0]
        + SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR_CCS_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2018"],
)

LCOA_SMR_CCS_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_CCS_initial_province_future[30, 9],
        C_variable_year=SMR_CCS_variable_province_future[30, 9],
        C_electricity_year=SMR_electricity_province[30]
        + SMR_CCS_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water + SMR_CCS_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock + SMR_CCS_chemical_feedstock,
        C_fixed_operation_year=SMR_CCS_fixed_operation_province_future[30, 9]
        + SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR_CCS_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2060"],
)
LCOA_SMR_CCS_results_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/4168953226.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SMR_transport_initial_future[i] * Carbon_transport_distance_province[j]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/4168953226.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + SMR_capture_initial_future[i]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/4168953226.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,4.752156,4.731422,4.679587,4.627752,4.573451,4.540729,4.527067,4.513405,4.501150,4.510275
Beijing,4.560522,4.539788,4.487953,4.436118,4.381817,4.349095,4.335433,4.321771,4.309515,4.318641
Chongqing,3.801102,3.780368,3.728533,3.676698,3.622397,3.589675,3.576013,3.562351,3.550096,3.559221
Fujian,5.716610,5.674101,5.567828,5.461555,5.352817,5.265657,5.245190,5.224723,5.192053,5.180765
Gansu,3.926606,3.905872,3.854037,3.802203,3.747902,3.715180,3.701518,3.687856,3.675600,3.684726
Guangdong,6.113794,6.071285,5.965013,5.858740,5.750001,5.662841,5.642374,5.621907,5.589238,5.577949
Guangxi,5.930789,5.888280,5.782007,5.675734,5.566996,5.479836,5.459369,5.438902,5.406232,5.394944
Guizhou,4.811848,4.791115,4.739280,4.687445,4.633144,4.600422,4.586760,4.573098,4.560842,4.569968
Hainan,4.890022,4.869288,4.817453,4.765618,4.711317,4.678595,4.664933,4.651271,4.639016,4.648141
Hebei,4.574368,4.553634,4.501800,4.449965,4.395664,4.362942,4.349280,4.335618,4.323362,4.332488


<IPython.core.display.Javascript object>

## PV

In [10]:
# PV
PV_data = pd.read_excel(excel_file, sheet_name="PV")

PV_initial_future = Future_data.iloc[2, 4:14] * 1000

Vr_PV_future = PV_initial_future * rv  # 固定资产残值
PV_variable_year_future = PV_initial_future * 0.03

PV_capacity_factor_province = [0] * 31
Q_PV_year_province = [0] * 31

for j in range(31):
    PV_capacity_factor_province[j] = Province_data.iloc[j + 2, 6]
    Q_PV_year_province[j] = 365 * 24 * PV_capacity_factor_province[j]

LCOE_PV_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_PV_results_province_future[j, i] = (
            PV_initial_future[i]
            + PV_variable_year_future[i] * a
            - Vr_PV_future[i] / (1 + r) ** lifetime
        ) / (Q_PV_year_province[j] * 1000 * a)


LCOE_PV_results_province_future_df = pd.DataFrame(
    LCOE_PV_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_PV_results_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3914487231.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  PV_initial_future[i]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3914487231.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + PV_variable_year_future[i] * a
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3914487231.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  - Vr_PV_future[i] / (1 + r) ** lifeti

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,0.375419,0.355265,0.304878,0.262057,0.233892,0.213022,0.200662,0.190734,0.184386,0.179659
Beijing,0.349050,0.330311,0.283463,0.243650,0.217463,0.198059,0.186567,0.177337,0.171434,0.167040
Chongqing,0.429972,0.406888,0.349180,0.300137,0.267878,0.243976,0.229820,0.218450,0.211179,0.205765
Fujian,0.337119,0.319021,0.273774,0.235322,0.210030,0.191289,0.180191,0.171276,0.165575,0.161330
Gansu,0.303769,0.287461,0.246691,0.212043,0.189252,0.172366,0.162365,0.154332,0.149195,0.145370
Guangdong,0.339051,0.320848,0.275343,0.236671,0.211233,0.192385,0.181223,0.172257,0.166524,0.162255
Guangxi,0.353600,0.334617,0.287158,0.246827,0.220298,0.200641,0.188999,0.179649,0.173669,0.169217
Guizhou,0.349612,0.330843,0.283920,0.244043,0.217813,0.198378,0.186868,0.177623,0.171711,0.167309
Hainan,0.301378,0.285199,0.244749,0.210374,0.187763,0.171009,0.161087,0.153117,0.148021,0.144226
Hebei,0.346818,0.328199,0.281651,0.242092,0.216072,0.196793,0.185375,0.176203,0.170338,0.165972


<IPython.core.display.Javascript object>

In [11]:
LCOE_PV_CAPEX_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_PV_CAPEX_results_province_future[j, i] = (
            PV_initial_future[i]
            #             + PV_variable_year_future[i] * a
            - Vr_PV_future[i] / (1 + r) ** lifetime
        ) / (Q_PV_year_province[j] * 1000 * a)


LCOE_PV_CAPEX_results_province_future_df = pd.DataFrame(
    LCOE_PV_CAPEX_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_PV_CAPEX_results_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2969132849.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  PV_initial_future[i]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2969132849.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  - Vr_PV_future[i] / (1 + r) ** lifetime


,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,0.280756,0.265683,0.228002,0.195979,0.174915,0.159308,0.150064,0.142640,0.137892,0.134357
Beijing,0.261035,0.247022,0.211987,0.182213,0.162629,0.148118,0.139524,0.132621,0.128207,0.124920
Chongqing,0.321553,0.304290,0.261133,0.224456,0.200332,0.182457,0.171870,0.163367,0.157930,0.153881
Fujian,0.252113,0.238579,0.204741,0.175985,0.157070,0.143055,0.134755,0.128088,0.123825,0.120650
Gansu,0.227173,0.214977,0.184487,0.158575,0.141532,0.128903,0.121424,0.115417,0.111575,0.108715
Guangdong,0.253558,0.239945,0.205914,0.176993,0.157970,0.143875,0.135527,0.128822,0.124534,0.121342
Guangxi,0.264438,0.250242,0.214750,0.184588,0.164749,0.150049,0.141343,0.134350,0.129878,0.126548
Guizhou,0.261456,0.247420,0.212328,0.182507,0.162891,0.148356,0.139748,0.132834,0.128413,0.125121
Hainan,0.225385,0.213285,0.183035,0.157327,0.140418,0.127889,0.120468,0.114508,0.110697,0.107859
Hebei,0.259367,0.245442,0.210632,0.181048,0.161589,0.147171,0.138632,0.131773,0.127387,0.124121


<IPython.core.display.Javascript object>

In [12]:
PV_Q_Green_year_province = [0] * 31
for j in range(31):
    PV_Q_Green_year_province[j] = 100000 * PV_capacity_factor_province[j]  # 绿氨年产能10万吨

PV_capacity_province = [0] * 31
for j in range(31):
    PV_capacity_province[j] = (
        PV_Q_Green_year_province[j]
        / (PV_capacity_factor_province[j])
        * (51.8 * 0.18878 + 0.11 * 0.87445 + 0.47475)
        / 8760
    )

PV_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_initial_province_future[j, i] = (
            Future_data.iloc[2, 4 + i] * 1000 * PV_capacity_province[j]
        )

PV_capacity_electrolyser_AWE_province = [0] * 31
for j in range(31):
    PV_capacity_electrolyser_AWE_province[j] = (
        (
            (PV_Q_Green_year_province[j] / (PV_capacity_factor_province[j]))
            * 0.18878
            * 1000
            / (1000 / 51.8)
        )
        / 8760
        * 1000
    )

PV_AWE_initial_electrolyser_stack_province_future = np.zeros((31, 10))
PV_AWE_initial_electrolyser_BoP_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_AWE_initial_electrolyser_stack_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / PV_capacity_electrolyser_AWE_province[j]) ** 0.88
        ) * 2.5
        PV_AWE_initial_electrolyser_BoP_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / PV_capacity_electrolyser_AWE_province[j]) ** 0.7
        )

PV_capacity_NH3_province = [0] * 31
PV_capacity_N2_province = [0] * 31
for j in range(31):
    PV_capacity_NH3_province[j] = (
        PV_Q_Green_year_province[j] / PV_capacity_factor_province[j]
    )
    PV_capacity_N2_province[j] = (
        0.87445 * PV_capacity_NH3_province[j]
    )  # 每千克合成氨需要投入0.87445kg氮气

initial_ASU_province = [0] * 31
initial_HB_province = [0] * 31

for j in range(31):
    initial_ASU_province[j] = PV_data.iloc[5, 2]
    initial_HB_province[j] = PV_data.iloc[6, 2]

PV_AWE_initial_province_future = np.zeros((31, 10))
PV_Vr_AWE_province_future = np.zeros((31, 10))
PV_AWE_non_PV_fixed_operation_province_future = np.zeros((31, 10))
PV_AWE_PV_fixed_operation_province_future = np.zeros((31, 10))
PV_AWE_fixed_operation_province_future = np.zeros((31, 10))

PV_AWE_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_AWE_initial_province_future[j, i] = (
            PV_AWE_initial_electrolyser_stack_province_future[j, i]
            + PV_AWE_initial_electrolyser_BoP_province_future[j, i]
            + initial_HB_province[j]
            + initial_ASU_province[j]
            + PV_initial_province_future[j, i]
        )
        PV_Vr_AWE_province_future[j, i] = (
            PV_AWE_initial_province_future[j, i] * rv
        )  # 固定资产残值
        PV_AWE_non_PV_fixed_operation_province_future[j, i] = 0.025 * (
            PV_AWE_initial_province_future[j, i] - PV_initial_province_future[j, i]
        )
        PV_AWE_PV_fixed_operation_province_future[j, i] = (
            0.03 * PV_initial_province_future[j, i]
        )
        PV_AWE_fixed_operation_province_future[j, i] = (
            PV_AWE_non_PV_fixed_operation_province_future[j, i]
            + PV_AWE_PV_fixed_operation_province_future[j, i]
        )

AWE_water = PV_data.iloc[121, 2] * PV_data.iloc[139, 2]
AWE_catalyst = PV_data.iloc[122, 2] * PV_data.iloc[140, 2]
AWE_chemical_feedstock = PV_data.iloc[123, 2] * PV_data.iloc[141, 2]

PV_AWE_electricity_province_future = np.zeros((31, 10))
PV_AWE_variable_operation_province_future = np.zeros((31, 10))
PV_AWE_variable_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        PV_AWE_electricity_province_future[j, i] = (
            PV_data.iloc[138, 2] * LCOE_PV_results_province_future[j, i]
        )

        PV_AWE_variable_operation_province_future[j, i] = (
            #            PV_AWE_electricity_province_future[j, i]
            +AWE_water
            + AWE_catalyst
            + AWE_chemical_feedstock
        )

        PV_AWE_variable_province_future[j, i] = (
            PV_AWE_fixed_operation_province_future[j, i]
        ) + (
            (PV_AWE_variable_operation_province_future[j, i])
            * PV_Q_Green_year_province[j]
            * 1000
        )

LCOA_PV_AWE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        LCOA_PV_AWE_results_province_future[j, i] = (
            PV_AWE_initial_province_future[j, i]
            + PV_AWE_variable_province_future[j, i] * a
            - PV_Vr_AWE_province_future[j, i] / (1 + r) ** lifetime
        ) / (PV_Q_Green_year_province[j] * 1000 * a)

LCOA_PV_AWE_results_province_future_df = pd.DataFrame(
    LCOA_PV_AWE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)


LCOA_PV_AWE_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=PV_AWE_initial_province_future[30, 0],
        C_variable_year=PV_AWE_variable_province_future[30, 0],
        C_electricity_year=0,  # PV_AWE_electricity_province_future[30, 0],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=PV_AWE_fixed_operation_province_future[30, 0],
        Q_year=PV_Q_Green_year_province[30],
        Vr=PV_Vr_AWE_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_PV_AWE_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=PV_AWE_initial_province_future[30, 9],
        C_variable_year=PV_AWE_variable_province_future[30, 9],
        C_electricity_year=0,  # PV_AWE_electricity_province_future[30, 9],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=PV_AWE_fixed_operation_province_future[30, 9],
        Q_year=PV_Q_Green_year_province[30],
        Vr=PV_Vr_AWE_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_PV_AWE_results_province_future_df

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,8.250243,7.681867,6.260929,4.918300,4.450992,4.059196,3.838932,3.643839,3.524608,3.422158
Beijing,7.672479,7.144027,5.822897,4.574576,4.140092,3.775817,3.571025,3.389635,3.278779,3.183525
Chongqing,9.445482,8.794515,7.167100,5.629372,5.094160,4.645432,4.393161,4.169719,4.033163,3.915826
Fujian,7.411089,6.900699,5.624724,4.419069,3.999436,3.647611,3.449818,3.274628,3.167562,3.075564
Gansu,6.680390,6.220491,5.070745,3.984361,3.606241,3.289221,3.110995,2.953136,2.856661,2.773764
Guangdong,7.453406,6.940092,5.656807,4.444244,4.022207,3.668367,3.469441,3.293247,3.185567,3.093042
Guangxi,7.772177,7.236836,5.898483,4.633888,4.193741,3.824716,3.617254,3.433500,3.321199,3.224703
Guizhou,7.684801,7.155497,5.832239,4.581906,4.146723,3.781860,3.576738,3.395056,3.284022,3.188614
Hainan,6.628008,6.171729,5.031031,3.953198,3.578054,3.263529,3.086706,2.930089,2.834374,2.752129
Hebei,7.623586,7.098513,5.785829,4.545488,4.113783,3.751836,3.548353,3.368123,3.257976,3.163331


<IPython.core.display.Javascript object>

In [13]:
LCOA_PV_AWE_results_2018

,2018
LCO,7.579714
CAPEX,5.780670
Electricity,0.000000
Fuel,0.000000
Energy_feedstock,0.000000
OPEX,1.799044


<IPython.core.display.Javascript object>

## Wind

In [14]:
# Wind
Wind_data = pd.read_excel(excel_file, sheet_name="Wind")

Wind_initial_future = Future_data.iloc[5, 4:14] * 1000

Vr_Wind_future = Wind_initial_future * rv  # 固定资产残值
Wind_variable_year_future = Wind_initial_future * 0.02

Wind_capacity_factor_province = [0] * 31
Q_Wind_year_province = [0] * 31

for j in range(31):
    Wind_capacity_factor_province[j] = Province_data.iloc[j + 2, 7]
    Q_Wind_year_province[j] = 365 * 24 * Wind_capacity_factor_province[j]

LCOE_Wind_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_Wind_results_province_future[j, i] = (
            Wind_initial_future[i]
            + Wind_variable_year_future[i] * a
            - Vr_Wind_future[i] / (1 + r) ** lifetime
        ) / (Q_Wind_year_province[j] * 1000 * a)
LCOE_Wind_results_province_future_df = pd.DataFrame(
    LCOE_Wind_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_Wind_results_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1310506303.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Wind_initial_future[i]
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1310506303.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + Wind_variable_year_future[i] * a
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1310506303.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  - Vr_Wind_future[i] / (1 + r) ** 

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,0.365654,0.359681,0.344749,0.329817,0.320549,0.311280,0.303351,0.295423,0.289505,0.283587
Beijing,0.470693,0.463004,0.443783,0.424562,0.412630,0.400699,0.390493,0.380287,0.372669,0.365051
Chongqing,0.464500,0.456912,0.437944,0.418975,0.407201,0.395427,0.385355,0.375283,0.367766,0.360248
Fujian,0.309666,0.304608,0.291963,0.279317,0.271467,0.263618,0.256903,0.250189,0.245177,0.240165
Gansu,0.335487,0.330007,0.316306,0.302606,0.294102,0.285598,0.278324,0.271050,0.265620,0.260190
Guangdong,0.421146,0.414267,0.397069,0.379871,0.369196,0.358520,0.349388,0.340257,0.333441,0.326625
Guangxi,0.368061,0.362049,0.347018,0.331988,0.322658,0.313328,0.305348,0.297367,0.291410,0.285454
Guizhou,0.416970,0.410159,0.393131,0.376103,0.365534,0.354964,0.345923,0.336882,0.330134,0.323385
Hainan,0.505376,0.497121,0.476483,0.455845,0.443035,0.430224,0.419266,0.408308,0.400129,0.391950
Hebei,0.346328,0.340671,0.326529,0.312386,0.303607,0.294828,0.287318,0.279809,0.274204,0.268599


<IPython.core.display.Javascript object>

In [15]:
Wind_Q_Green_year_province = [0] * 31
for j in range(31):
    Wind_Q_Green_year_province[j] = 100000 * Wind_capacity_factor_province[j]  # 绿氨年产能10万吨

Wind_capacity_province = [0] * 31
for j in range(31):
    Wind_capacity_province[j] = (
        Wind_Q_Green_year_province[j]
        / (Wind_capacity_factor_province[j])
        * (51.8 * 0.18878 + 0.11 * 0.87445 + 0.47475)
        / 8760
    )

Wind_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_initial_province_future[j, i] = (
            Future_data.iloc[5, 4 + i] * 1000 * Wind_capacity_province[j]
        )

Wind_capacity_electrolyser_AWE_province = [0] * 31
for j in range(31):
    Wind_capacity_electrolyser_AWE_province[j] = (
        (
            (Wind_Q_Green_year_province[j] / (Wind_capacity_factor_province[j]))
            * 0.18878
            * 1000
            / (1000 / 51.8)
        )
        / 8760
        * 1000
    )

Wind_AWE_initial_electrolyser_stack_province_future = np.zeros((31, 10))
Wind_AWE_initial_electrolyser_BoP_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_AWE_initial_electrolyser_stack_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / Wind_capacity_electrolyser_AWE_province[j]) ** 0.88
        ) * 2.5
        Wind_AWE_initial_electrolyser_BoP_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / Wind_capacity_electrolyser_AWE_province[j]) ** 0.7
        )

Wind_capacity_NH3_province = [0] * 31
Wind_capacity_N2_province = [0] * 31
for j in range(31):
    Wind_capacity_NH3_province[j] = (
        Wind_Q_Green_year_province[j] / Wind_capacity_factor_province[j]
    )
    Wind_capacity_N2_province[j] = (
        0.87445 * Wind_capacity_NH3_province[j]
    )  # 每千克合成氨需要投入0.87445kg氮气

initial_ASU_province = [0] * 31
initial_HB_province = [0] * 31

for j in range(31):
    initial_ASU_province[j] = Wind_data.iloc[5, 2]
    initial_HB_province[j] = Wind_data.iloc[6, 2]

Wind_AWE_initial_province_future = np.zeros((31, 10))
Wind_Vr_AWE_province_future = np.zeros((31, 10))
Wind_AWE_non_Wind_fixed_operation_province_future = np.zeros((31, 10))
Wind_AWE_Wind_fixed_operation_province_future = np.zeros((31, 10))
Wind_AWE_fixed_operation_province_future = np.zeros((31, 10))

Wind_AWE_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_AWE_initial_province_future[j, i] = (
            Wind_AWE_initial_electrolyser_stack_province_future[j, i]
            + Wind_AWE_initial_electrolyser_BoP_province_future[j, i]
            + initial_HB_province[j]
            + initial_ASU_province[j]
            + Wind_initial_province_future[j, i]
        )
        Wind_Vr_AWE_province_future[j, i] = (
            Wind_AWE_initial_province_future[j, i] * rv
        )  # 固定资产残值
        Wind_AWE_non_Wind_fixed_operation_province_future[j, i] = 0.025 * (
            Wind_AWE_initial_province_future[j, i] - Wind_initial_province_future[j, i]
        )
        Wind_AWE_Wind_fixed_operation_province_future[j, i] = (
            0.02 * Wind_initial_province_future[j, i]
        )
        Wind_AWE_fixed_operation_province_future[j, i] = (
            Wind_AWE_non_Wind_fixed_operation_province_future[j, i]
            + Wind_AWE_Wind_fixed_operation_province_future[j, i]
        )

AWE_water = Wind_data.iloc[121, 2] * Wind_data.iloc[139, 2]
AWE_catalyst = Wind_data.iloc[122, 2] * Wind_data.iloc[140, 2]
AWE_chemical_feedstock = Wind_data.iloc[123, 2] * Wind_data.iloc[141, 2]

Wind_AWE_electricity_province_future = np.zeros((31, 10))
Wind_AWE_variable_operation_province_future = np.zeros((31, 10))
Wind_AWE_variable_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        Wind_AWE_electricity_province_future[j, i] = (
            Wind_data.iloc[138, 2] * LCOE_Wind_results_province_future[j, i]
        )

        Wind_AWE_variable_operation_province_future[j, i] = (
            #            Wind_AWE_electricity_province_future[j, i]
            +AWE_water
            + AWE_catalyst
            + AWE_chemical_feedstock
        )

        Wind_AWE_variable_province_future[j, i] = (
            Wind_AWE_fixed_operation_province_future[j, i]
        ) + (
            (Wind_AWE_variable_operation_province_future[j, i])
            * Wind_Q_Green_year_province[j]
            * 1000
        )

LCOA_Wind_AWE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        LCOA_Wind_AWE_results_province_future[j, i] = (
            Wind_AWE_initial_province_future[j, i]
            + Wind_AWE_variable_province_future[j, i] * a
            - Wind_Vr_AWE_province_future[j, i] / (1 + r) ** lifetime
        ) / (Wind_Q_Green_year_province[j] * 1000 * a)

LCOA_Wind_AWE_results_province_future_df = pd.DataFrame(
    LCOA_Wind_AWE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)


LCOA_Wind_AWE_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=Wind_AWE_initial_province_future[30, 0],
        C_variable_year=Wind_AWE_variable_province_future[30, 0],
        C_electricity_year=0,  # Wind_AWE_electricity_province_future[30, 0],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=Wind_AWE_fixed_operation_province_future[30, 0],
        Q_year=Wind_Q_Green_year_province[30],
        Vr=Wind_Vr_AWE_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_Wind_AWE_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=Wind_AWE_initial_province_future[30, 9],
        C_variable_year=Wind_AWE_variable_province_future[30, 9],
        C_electricity_year=0,  # Wind_AWE_electricity_province_future[30, 9],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=Wind_AWE_fixed_operation_province_future[30, 9],
        Q_year=Wind_Q_Green_year_province[30],
        Vr=Wind_Vr_AWE_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_Wind_AWE_results_province_future_df


,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,7.052651,6.721956,5.895220,5.068485,4.841173,4.613862,4.462793,4.311724,4.210472,4.109221
Beijing,9.071484,8.645793,7.581567,6.517340,6.224731,5.932122,5.737656,5.543190,5.412853,5.282515
Chongqing,8.952449,8.532359,7.482136,6.431912,6.143153,5.854394,5.662487,5.470580,5.341957,5.213335
Fujian,5.976575,5.696515,4.996366,4.296217,4.103711,3.911205,3.783267,3.655329,3.569581,3.483833
Gansu,6.472833,6.169422,5.410895,4.652367,4.443810,4.235252,4.096647,3.958041,3.865143,3.772245
Guangdong,8.119204,7.738323,6.786120,5.833918,5.572109,5.310301,5.136305,4.962309,4.845692,4.729075
Guangxi,7.098904,6.766034,5.933856,5.101679,4.872872,4.644065,4.492001,4.339938,4.238020,4.136102
Guizhou,8.038925,7.661821,6.719062,5.776304,5.517092,5.257880,5.085610,4.913340,4.797879,4.682418
Hainan,9.738079,9.281022,8.138379,6.995736,6.681566,6.367396,6.158601,5.949806,5.809865,5.669924
Hebei,6.681214,6.367997,5.584956,4.801915,4.586618,4.371321,4.228236,4.085151,3.989251,3.893351


<IPython.core.display.Javascript object>

In [16]:
LCOA_Wind_AWE_results_2018

,2018
LCO,7.241205
CAPEX,5.772681
Electricity,0.000000
Fuel,0.000000
Energy_feedstock,0.000000
OPEX,1.468524


<IPython.core.display.Javascript object>

In [17]:
LCOA_Wind_AWE_results_2060

,2060
LCO,4.218804
CAPEX,3.381992
Electricity,0.000000
Fuel,0.000000
Energy_feedstock,0.000000
OPEX,0.836811


<IPython.core.display.Javascript object>

# integrated

In [18]:
# electricity
PV_capacity_province_future = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\capacity_PV.xlsx"
).iloc[:, 1:]
Wind_capacity_province_future = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\capacity_Wind.xlsx"
).iloc[:, 1:]

PV_cost_province_future = np.zeros((30, 10))
Wind_cost_province_future = np.zeros((30, 10))
Vr_PV_cost_province_future = np.zeros((30, 10))
Vr_Wind_cost_province_future = np.zeros((30, 10))
PV_variable_year_future = np.zeros((30, 10))
Wind_variable_year_future = np.zeros((30, 10))

for i in range(10):
    for j in range(30):
        PV_cost_province_future[j, i] = (
            PV_initial_future[i] * PV_capacity_province_future.iloc[j, i] / 1000
        )
        Wind_cost_province_future[j, i] = (
            Wind_initial_future[i] * Wind_capacity_province_future.iloc[j, i] / 1000
        )
        Vr_PV_cost_province_future[j, i] = PV_cost_province_future[j, i] * rv
        Vr_Wind_cost_province_future[j, i] = PV_cost_province_future[j, i] * rv
        PV_variable_year_future[j, i] = PV_cost_province_future[j, i] * 0.03
        Wind_variable_year_future[j, i] = Wind_cost_province_future[j, i] * 0.02

Q_electricity = 100000000 * (0.18878 * 51.8 + 0.877445 * 1.1 + 0.47473)

LCOE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(30):
        LCOE_results_province_future[j, i] = (
            PV_cost_province_future[j, i]
            + Wind_cost_province_future[j, i]
            + (PV_variable_year_future[j, i] + Wind_variable_year_future[j, i]) * a
            - (Vr_PV_cost_province_future[j, i] + Vr_Wind_cost_province_future[j, i])
            / (1 + r) ** lifetime
        ) / (Q_electricity * a)
LCOE_results_province_future[30, :] = np.mean(
    LCOE_results_province_future[:30, :], axis=0
)

electricity_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        electricity_province_future[j, i] = LCOE_results_province_future[j, i] * (
            0.18878 * 51.8 + 0.877445 * 0.11 + 0.47473
        )

LCOE_results_province_future_df = pd.DataFrame(
    LCOE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
electricity_province_future_df = pd.DataFrame(
    electricity_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
electricity_province_future_df

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3286556343.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  PV_initial_future[i] * PV_capacity_province_future.iloc[j, i] / 1000
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3286556343.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Wind_initial_future[i] * Wind_capacity_province_future.iloc[j, i] / 1000


,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
Anhui,3.855220,3.731074,3.387567,2.996605,2.754725,2.148167,2.013507,1.920029,1.859397,1.810349
Beijing,3.764867,3.403115,2.820706,2.400668,2.146091,1.954601,1.835320,1.747011,1.691889,1.647943
Chongqing,4.689866,4.532161,4.060770,3.476138,2.686509,2.446028,2.292254,2.187551,2.117217,2.062083
Fujian,3.300817,3.225121,3.001463,2.730316,2.539234,2.357616,2.214823,2.124742,2.061491,1.982665
Gansu,3.438744,3.315556,2.890396,2.115538,1.890887,1.722156,1.614528,1.539459,1.490653,1.451957
Guangdong,4.036785,3.873340,3.377025,2.403421,2.147215,1.955542,1.831498,1.748664,1.692665,1.648580
Guangxi,3.809907,3.664310,3.276135,2.853001,2.232936,2.033480,1.904513,1.815210,1.760126,1.713182
Guizhou,4.143431,4.004731,3.574209,2.532326,2.268797,2.066356,1.927814,1.841443,1.788462,1.739614
Hainan,3.094743,2.919524,2.482997,2.096849,1.875977,1.708794,1.599430,1.526028,1.480255,1.440157
Hebei,3.661971,3.543058,3.191759,2.598349,2.139752,1.948571,1.829536,1.742993,1.686631,1.642855


<IPython.core.display.Javascript object>

In [19]:
LCOA_CG_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\1 LCOA_CG_results_province_future_df.xlsx"
)
LCOA_SMR_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\2 LCOA_SMR_results_province_future_df.xlsx"
)
LCOA_CG_CCS_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\3 LCOA_CG_CCS_results_province_future_df.xlsx"
)
LCOA_SMR_CCS_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\4 LCOA_SMR_CCS_results_province_future_df.xlsx"
)
LCOA_PV_AWE_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\5 LCOA_PV_AWE_results_province_future_df.xlsx"
)
LCOA_Wind_AWE_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\6 LCOA_Wind_AWE_results_province_future_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/514418988.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_CG_results_province_future_df.to_excel(
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/514418988.py:4: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_SMR_results_province_future_df.to_excel(
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/514418988.py:7: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_CG_CCS_results_province_future_df.to_excel(
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/514418988.py:10: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_SMR_CCS_results_province_future_df.to_excel(
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/514418988.py:13: UserWarning: Pandas requires 

<IPython.core.display.Javascript object>

In [20]:
LCOE_PV_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\8 LCOE_PV_results_province_future_df.xlsx"
)
LCOE_Wind_results_province_future_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\Province\9 LCOE_Wind_results_province_future_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1549639972.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOE_PV_results_province_future_df.to_excel(
C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1549639972.py:4: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOE_Wind_results_province_future_df.to_excel(


<IPython.core.display.Javascript object>

In [21]:
LCOA_2018 = np.zeros((6, 6))
LCOA_2018[:, 0] = LCOA_CG_results_2018.iloc[:, 0]
LCOA_2018[:, 2] = LCOA_CG_CCS_results_2018.iloc[:, 0]
LCOA_2018[:, 1] = LCOA_SMR_results_2018.iloc[:, 0]
LCOA_2018[:, 3] = LCOA_SMR_CCS_results_2018.iloc[:, 0]
LCOA_2018[:, 4] = LCOA_PV_AWE_results_2018.iloc[:, 0]
LCOA_2018[:, 5] = LCOA_Wind_AWE_results_2018.iloc[:, 0]
LCOA_2018_df = pd.DataFrame(
    LCOA_2018,
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind"],
)
LCOA_2018_df


,CG,SMR,CG+CCS,SMR+CCS,PV,Wind
LCO,3.570909,4.144863,5.191231,4.637329,7.579714,7.241205
CAPEX,1.539806,0.854510,2.710241,1.283303,5.780670,5.772681
Electricity,0.189131,0.528569,0.191825,0.529556,0.000000,0.000000
Fuel,0.191577,0.240530,0.191577,0.240530,0.000000,0.000000
Energy_feedstock,0.863961,2.240552,0.863961,2.240552,0.000000,0.000000
OPEX,0.786434,0.280702,1.233626,0.343388,1.799044,1.468524


<IPython.core.display.Javascript object>

In [22]:
LCOA_2018_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\1 LCOA_2018_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/462560754.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_2018_df.to_excel(


<IPython.core.display.Javascript object>

In [23]:
LCOA_2060 = np.zeros((6, 6))
LCOA_2060[:, 0] = LCOA_CG_results_2060.iloc[:, 0]
LCOA_2060[:, 2] = LCOA_CG_CCS_results_2060.iloc[:, 0]
LCOA_2060[:, 1] = LCOA_SMR_results_2060.iloc[:, 0]
LCOA_2060[:, 3] = LCOA_SMR_CCS_results_2060.iloc[:, 0]
LCOA_2060[:, 4] = LCOA_PV_AWE_results_2060.iloc[:, 0]
LCOA_2060[:, 5] = LCOA_Wind_AWE_results_2060.iloc[:, 0]
LCOA_2060_df = pd.DataFrame(
    LCOA_2060,
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind"],
)
LCOA_2060_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind
LCO,3.570909,4.144863,4.174319,4.366052,3.145211,4.218804
CAPEX,1.539806,0.854510,1.967481,1.011190,2.379909,3.381992
Electricity,0.189131,0.528569,0.191825,0.529556,0.000000,0.000000
Fuel,0.191577,0.240530,0.191577,0.240530,0.000000,0.000000
Energy_feedstock,0.863961,2.240552,0.863961,2.240552,0.000000,0.000000
OPEX,0.786434,0.280702,0.959474,0.344224,0.765301,0.836811


<IPython.core.display.Javascript object>

In [24]:
LCOA_2060_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\2 LCOA_2060_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3914319057.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_2060_df.to_excel(


<IPython.core.display.Javascript object>

In [25]:
LCOA_integrated_results_province_future_18_60_df = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\integrated_cost.xlsx"
).iloc[:, 1:]
LCOA_integrated_results_province_future_18_60_df

,2018,2020,2025,2030,2035,2040,2045,2050,2055,2060
0,5.650704,5.422661,4.822809,4.188704,3.914886,3.628124,3.322995,3.214129,3.144350,3.054433
1,6.416756,6.026023,4.966798,3.978304,3.586661,3.267957,2.995060,2.895974,2.832529,2.751764
2,6.410485,6.175776,5.538386,4.825473,4.372598,3.983623,3.655567,3.531552,3.452076,3.354024
3,4.769069,4.604996,4.168027,3.692444,3.493035,3.297603,3.106712,3.023051,2.965091,2.889966
4,5.025077,4.835760,4.293372,3.578521,3.227598,2.940988,2.692697,2.605423,2.549477,2.476561
5,5.957844,5.703223,5.017202,4.111154,3.708459,3.378795,3.091158,2.992055,2.928519,2.844440
6,5.665244,5.427222,4.794657,4.121098,3.790663,3.453674,3.162296,3.059352,2.993327,2.907671
7,5.967833,5.733571,5.109133,4.304206,3.881927,3.536755,3.236670,3.132321,3.065264,2.977395
8,5.997433,5.598073,4.598114,3.659925,3.302820,3.009427,2.750571,2.664196,2.608775,2.533649
9,5.358591,5.151594,4.586011,3.923667,3.558929,3.242712,2.972831,2.873979,2.810668,2.730625


<IPython.core.display.Javascript object>

In [26]:
# 每五年线性平均至每年
def calculate_lcoa_results(input_df):
    lcoa_results = np.zeros((31, 43))
    lcoa_results[:, 0] = input_df.iloc[:, 0]
    lcoa_results[:, 1] = (input_df.iloc[:, 0] + input_df.iloc[:, 1]) / 2

    for a in range(8):
        for i in range(5):
            yearly_average = (input_df.iloc[:, a + 2] - input_df.iloc[:, a + 1]) / 5
            lcoa_results[:, i + 5 * a + 2] = (
                yearly_average * i + input_df.iloc[:, a + 1]
            )

    lcoa_results[:, 42] = input_df.iloc[:, 9]

    return pd.DataFrame(lcoa_results)


# Calculate LCOA results for different dataframes
LCOA_CG_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_CG_results_province_future_df
)
LCOA_CG_CCS_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_CG_CCS_results_province_future_df
)
LCOA_SMR_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_SMR_results_province_future_df
)
LCOA_SMR_CCS_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_SMR_CCS_results_province_future_df
)
LCOA_PV_AWE_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_PV_AWE_results_province_future_df
)
LCOA_Wind_AWE_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_Wind_AWE_results_province_future_df
)
LCOA_integrated_results_province_future_18_60 = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\integrated_cost.xlsx"
).iloc[0:31, 1:]
LCOA_integrated_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_integrated_results_province_future_18_60
)
LCOA_18_60 = np.zeros((31, 301))

for i in range(43):
    LCOA_18_60[:, 0 + i * 7] = LCOA_CG_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 2 + i * 7] = LCOA_CG_CCS_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 1 + i * 7] = LCOA_SMR_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 3 + i * 7] = LCOA_SMR_CCS_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 4 + i * 7] = LCOA_PV_AWE_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 5 + i * 7] = LCOA_Wind_AWE_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 6 + i * 7] = LCOA_integrated_results_province_future_18_60_df.iloc[
        :, i
    ]

LCOA_18_60_df = pd.DataFrame(LCOA_18_60, index=Province)

LCOA_18_60_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
Anhui,3.724672,4.327076,5.161054,4.752156,8.250243,7.052651,5.650704,3.724672,4.327076,5.123959,...,3.442648,4.129471,3.072416,3.724672,4.327076,4.224383,4.510275,3.422158,4.109221,3.054433
Beijing,3.577310,4.135376,5.013872,4.560522,7.672479,9.071484,6.416756,3.577310,4.135376,4.976778,...,3.202576,5.308583,2.767917,3.577310,4.135376,4.077201,4.318641,3.183525,5.282515,2.751764
Chongqing,3.692834,3.376092,5.129026,3.801102,9.445482,8.952449,6.410485,3.692834,3.376092,5.091931,...,3.939293,5.239060,3.373634,3.692834,3.376092,4.192355,3.559221,3.915826,5.213335,3.354024
Fujian,3.709195,4.616607,6.987851,5.716610,7.411089,5.976575,4.769069,3.709195,4.616607,6.921037,...,3.093963,3.500982,2.904991,3.709195,4.616607,5.248773,5.180765,3.075564,3.483833,2.889966
Gansu,3.442416,3.501823,4.877989,3.926606,6.680390,6.472833,5.025077,3.442416,3.501823,4.840894,...,2.790344,3.790825,2.491144,3.442416,3.501823,3.941317,3.684726,2.773764,3.772245,2.476561
Guangdong,3.749437,5.013750,7.028205,6.113794,7.453406,8.119204,5.957844,3.749437,5.013750,6.961391,...,3.111547,4.752398,2.861256,3.749437,5.013750,5.289127,5.577949,3.093042,4.729075,2.844440
Guangxi,3.945035,4.830713,7.223889,5.930789,7.772177,7.098904,5.665244,3.945035,4.830713,7.157075,...,3.244002,4.156486,2.924802,3.945035,4.830713,5.484811,5.394944,3.224703,4.136102,2.907671
Guizhou,3.464748,4.386944,4.900653,4.811848,7.684801,8.038925,5.967833,3.464748,4.386944,4.863558,...,3.207696,4.705510,2.994969,3.464748,4.386944,3.963982,4.569968,3.188614,4.682418,2.977395
Hainan,3.663596,4.464924,5.100027,4.890022,6.628008,9.738079,5.997433,3.663596,4.464924,5.062932,...,2.768578,5.697912,2.548674,3.663596,4.464924,4.163356,4.648141,2.752129,5.669924,2.533649
Hebei,3.471977,4.149443,4.907938,4.574368,7.623586,6.681214,5.358591,3.471977,4.149443,4.870843,...,3.182260,3.912531,2.746634,3.471977,4.149443,3.971266,4.332488,3.163331,3.893351,2.730625


<IPython.core.display.Javascript object>

In [27]:
LCOA_18_60_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\20 LCOA_18_60_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2476491187.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_18_60_df.to_excel(


<IPython.core.display.Javascript object>

In [28]:
LCOA_df_national = np.zeros((7, 43))
for i in range(43):
    for z in range(7):
        for j in range(7):
            LCOA_df_national[j, i] = LCOA_18_60_df.iloc[30, j + (z + 1) * i]
LCOA_df_national_df = pd.DataFrame(
    np.transpose(LCOA_df_national),
    index=Year1,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_df_national_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
2018,3.570909,4.144863,5.191231,4.637329,7.579714,7.241205,5.607758
2019,3.570909,4.144863,5.151164,4.625873,7.318693,7.071421,5.484301
2020,3.570909,4.144863,5.111097,4.614417,7.057672,6.901638,5.360844
2021,3.570909,4.144863,5.071030,4.602962,6.796651,6.731855,5.227791
2022,3.570909,4.144863,5.030964,4.591506,6.535630,6.562071,5.094738
2023,3.570909,4.144863,4.990897,4.580050,6.274609,6.392288,4.961685
2024,3.570909,4.144863,4.950830,4.568595,6.013588,6.222504,4.828632
2025,3.570909,4.144863,4.910763,4.557139,5.752567,6.052721,4.695579
2026,3.570909,4.144863,4.870696,4.545683,5.505932,5.882938,4.549538
2027,3.570909,4.144863,4.830629,4.534227,5.259296,5.713154,4.403497


<IPython.core.display.Javascript object>

In [29]:
LCOA_df_national_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\7 LCOA_df_national_18_60_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/518761019.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_df_national_df.to_excel(


<IPython.core.display.Javascript object>

In [30]:
# 求出每年不同制氨技术成本的最小值 - 不加碳成本

LCOA_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_18_60_min[j, i] = np.min(LCOA_18_60_df.iloc[j, (7 * i) : (7 * i + 7)])
LCOA_18_60_min_df = pd.DataFrame(LCOA_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_18_60_min_sum = pd.DataFrame(
    (LCOA_18_60_min_df.values.sum(axis=1) / 43), index=Province, columns=["min"]
)
LCOA_18_60_min_sum

,min
Anhui,3.496712
Beijing,3.250646
Chongqing,3.375522
Fujian,3.341512
Gansu,3.008212
Guangdong,3.380273
Guangxi,3.497520
Guizhou,3.322190
Hainan,3.121461
Hebei,3.192618


<IPython.core.display.Javascript object>

In [31]:
LCOA_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if LCOA_18_60_df.iloc[j, 7 * i + 0] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "A"
        elif LCOA_18_60_df.iloc[j, 7 * i + 1] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "B"
        elif LCOA_18_60_df.iloc[j, 7 * i + 2] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "C"
        elif LCOA_18_60_df.iloc[j, 7 * i + 3] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "D"
        elif LCOA_18_60_df.iloc[j, 7 * i + 4] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "E"
        elif LCOA_18_60_df.iloc[j, 7 * i + 5] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "F"
        elif LCOA_18_60_df.iloc[j, 7 * i + 6] == LCOA_18_60_min_df.iloc[j, i]:
            LCOA_18_60_min_type[j, i] = "G"
LCOA_18_60_min_type_df = pd.DataFrame(
    LCOA_18_60_min_type, index=Province, columns=Year1
)
LCOA_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Beijing,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Chongqing,B,B,B,B,B,B,B,B,B,B,...,B,B,B,B,B,B,B,B,G,G
Fujian,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Gansu,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guangdong,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guangxi,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guizhou,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Hainan,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Hebei,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G


<IPython.core.display.Javascript object>

In [32]:
LCOA_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "A"
    )
    LCOA_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "B"
    )
    LCOA_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "C"
    )
    LCOA_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "D"
    )
    LCOA_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "E"
    )
    LCOA_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "F"
    )
    LCOA_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_18_60_min_type[j, :] == "G"
    )
LCOA_18_60_min_type_province_df = pd.DataFrame(
    LCOA_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,21.0,0.0,0.0,0.0,0.0,0.0,22.0
Beijing,18.0,0.0,0.0,0.0,0.0,0.0,25.0
Chongqing,0.0,41.0,0.0,0.0,0.0,0.0,2.0
Fujian,12.0,0.0,0.0,0.0,0.0,0.0,31.0
Gansu,14.0,0.0,0.0,0.0,0.0,0.0,29.0
Guangdong,17.0,0.0,0.0,0.0,0.0,0.0,26.0
Guangxi,15.0,0.0,0.0,0.0,0.0,0.0,28.0
Guizhou,24.0,0.0,0.0,0.0,0.0,0.0,19.0
Hainan,12.0,0.0,0.0,0.0,0.0,0.0,31.0
Hebei,19.0,0.0,0.0,0.0,0.0,0.0,24.0


<IPython.core.display.Javascript object>

In [33]:
LCOA_18_60_min_type_province_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\3 LCOA_18_60_min_type_province_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/4255559107.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_18_60_min_type_province_df.to_excel(


<IPython.core.display.Javascript object>

In [34]:
# 求出每年传统制氨技术成本的最小值 - 不加碳成本

LCOA_conventional_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_conventional_18_60_min[j, i] = np.min(
            LCOA_18_60_df.iloc[j, (7 * i) : (7 * i + 2)]
        )
LCOA_conventional_18_60_min_df = pd.DataFrame(LCOA_conventional_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_conventional_18_60_min_sum = pd.DataFrame(
    (LCOA_conventional_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_conventional"],
)
LCOA_conventional_18_60_min_sum

,min_conventional
Anhui,3.724672
Beijing,3.577310
Chongqing,3.376092
Fujian,3.709195
Gansu,3.442416
Guangdong,3.749437
Guangxi,3.945035
Guizhou,3.464748
Hainan,3.663596
Hebei,3.471977


<IPython.core.display.Javascript object>

In [35]:
LCOA_conventional_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if (
            LCOA_18_60_df.iloc[j, 7 * i + 0]
            == LCOA_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_conventional_18_60_min_type[j, i] = "A"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 1]
            == LCOA_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_conventional_18_60_min_type[j, i] = "B"

LCOA_conventional_18_60_min_type_df = pd.DataFrame(
    LCOA_conventional_18_60_min_type, index=Province, columns=Year1
)
LCOA_conventional_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Beijing,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Chongqing,B,B,B,B,B,B,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Fujian,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Gansu,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Guangdong,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Guangxi,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Guizhou,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Hainan,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Hebei,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A


<IPython.core.display.Javascript object>

In [36]:
LCOA_conventional_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_conventional_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_conventional_18_60_min_type[j, :] == "A"
    )
    LCOA_conventional_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_conventional_18_60_min_type[j, :] == "B"
    )
LCOA_conventional_18_60_min_type_province_df = pd.DataFrame(
    LCOA_conventional_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_conventional_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Beijing,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Chongqing,0.0,43.0,0.0,0.0,0.0,0.0,0.0
Fujian,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Gansu,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Guangdong,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Guangxi,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Guizhou,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Hainan,43.0,0.0,0.0,0.0,0.0,0.0,0.0
Hebei,43.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [37]:
# 求出每年低碳氨技术成本的最小值 - 不加碳成本

LCOA_low_carbon_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_low_carbon_18_60_min[j, i] = np.min(
            LCOA_18_60_df.iloc[j, (7 * i + 2) : (7 * i + 7)]
        )
LCOA_low_carbon_18_60_min_df = pd.DataFrame(LCOA_low_carbon_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_low_carbon_18_60_min_sum = pd.DataFrame(
    (LCOA_low_carbon_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_low_carbon"],
)
LCOA_low_carbon_18_60_min_sum

,min_low_carbon
Anhui,3.804071
Beijing,3.529524
Chongqing,3.603084
Fujian,3.503972
Gansu,3.130003
Guangdong,3.803909
Guangxi,3.792825
Guizhou,3.773181
Hainan,3.380386
Hebei,3.501919


<IPython.core.display.Javascript object>

In [38]:
LCOA_two_type_18_60_min_sum = pd.concat(
    [LCOA_conventional_18_60_min_sum, LCOA_low_carbon_18_60_min_sum], axis=1,
)
LCOA_two_type_18_60_min_sum

,min_conventional,min_low_carbon
Anhui,3.724672,3.804071
Beijing,3.577310,3.529524
Chongqing,3.376092,3.603084
Fujian,3.709195,3.503972
Gansu,3.442416,3.130003
Guangdong,3.749437,3.803909
Guangxi,3.945035,3.792825
Guizhou,3.464748,3.773181
Hainan,3.663596,3.380386
Hebei,3.471977,3.501919


<IPython.core.display.Javascript object>

In [39]:
LCOA_two_type_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\4 LCOA_two_type_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3805699932.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_two_type_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [40]:
LCOA_low_carbon_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if LCOA_18_60_df.iloc[j, 7 * i + 0] == LCOA_low_carbon_18_60_min_df.iloc[j, i]:
            LCOA_low_carbon_18_60_min_type[j, i] = "A"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 1] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "B"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 2] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "C"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 3] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "D"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 4] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "E"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 5] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "F"
        elif (
            LCOA_18_60_df.iloc[j, 7 * i + 6] == LCOA_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_low_carbon_18_60_min_type[j, i] = "G"
LCOA_low_carbon_18_60_min_type_df = pd.DataFrame(
    LCOA_low_carbon_18_60_min_type, index=Province, columns=Year1
)
LCOA_low_carbon_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,D,D,D,D,D,D,D,D,D,G,...,G,G,G,G,G,G,G,G,G,G
Beijing,D,D,D,D,D,D,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Chongqing,D,D,D,D,D,D,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Fujian,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Gansu,D,D,D,D,D,D,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Guangdong,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guangxi,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guizhou,D,D,D,D,C,C,C,C,C,C,...,G,G,G,G,G,G,G,G,G,G
Hainan,D,D,D,D,D,D,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Hebei,D,D,D,D,D,D,D,D,G,G,...,G,G,G,G,G,G,G,G,G,G


<IPython.core.display.Javascript object>

In [41]:
LCOA_low_carbon_18_60_min_type_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\21 LCOA_low_carbon_18_60_min_type_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3503785008.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_low_carbon_18_60_min_type_df.to_excel(


<IPython.core.display.Javascript object>

In [42]:
LCOA_low_carbon_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_low_carbon_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_low_carbon_18_60_min_type[j, :] == "C"
    )
    LCOA_low_carbon_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_low_carbon_18_60_min_type[j, :] == "D"
    )
    LCOA_low_carbon_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_low_carbon_18_60_min_type[j, :] == "E"
    )
    LCOA_low_carbon_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_low_carbon_18_60_min_type[j, :] == "F"
    )
    LCOA_low_carbon_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_low_carbon_18_60_min_type[j, :] == "G"
    )
LCOA_low_carbon_18_60_min_type_province_df = pd.DataFrame(
    LCOA_low_carbon_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_low_carbon_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,0.0,0.0,0.0,9.0,0.0,0.0,34.0
Beijing,0.0,0.0,0.0,10.0,0.0,0.0,33.0
Chongqing,0.0,0.0,0.0,31.0,0.0,0.0,12.0
Fujian,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Gansu,0.0,0.0,0.0,11.0,0.0,0.0,32.0
Guangdong,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Guangxi,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Guizhou,0.0,0.0,7.0,4.0,0.0,0.0,32.0
Hainan,0.0,0.0,0.0,6.0,0.0,0.0,37.0
Hebei,0.0,0.0,0.0,8.0,0.0,0.0,35.0


<IPython.core.display.Javascript object>

In [43]:
carbon_cost_df = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\2 LCA\Carbon_cost.xlsx", "Carbon cost",
)
carbon_cost = [0] * 10
for i in range(9):
    carbon_cost[i + 1] = carbon_cost_df.iloc[21, i + 2] / 1000
carbon_cost

[0,
 0.12531961090909088,
 0.3077599636363637,
 0.3570908727272727,
 0.3960159272727273,
 0.43797130909090914,
 0.49339625454545455,
 0.627079309090909,
 1.078358836363636,
 1.7797683272727272]

<IPython.core.display.Javascript object>

In [44]:
LCA_df = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\2 LCA\LCA.xlsx", "LCA",
)

CG_LCA = LCA_df.iloc[35:66, 13:23]
SMR_LCA = LCA_df.iloc[105:136, 13:23]
CG_CCS_LCA = LCA_df.iloc[70:101, 13:23]
SMR_CCS_LCA = LCA_df.iloc[140:171, 13:23]
PV_AWE_LCA = LCA_df.iloc[175:206, 1:11]
Wind_AWE_LCA = LCA_df.iloc[210:241, 1:11]
Integrated_AWE_LCA = LCA_df.iloc[246:277, 1:11]

CG_carbon_cost = np.zeros((31, 10))
SMR_carbon_cost = np.zeros((31, 10))
CG_CCS_carbon_cost = np.zeros((31, 10))
SMR_CCS_carbon_cost = np.zeros((31, 10))
PV_carbon_cost = np.zeros((31, 10))
Wind_carbon_cost = np.zeros((31, 10))
Integrated_carbon_cost = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        CG_carbon_cost[j, i] = CG_LCA.iloc[j, i] * carbon_cost[i]
        SMR_carbon_cost[j, i] = SMR_LCA.iloc[j, i] * carbon_cost[i]
        CG_CCS_carbon_cost[j, i] = CG_CCS_LCA.iloc[j, i] * carbon_cost[i]
        SMR_CCS_carbon_cost[j, i] = SMR_CCS_LCA.iloc[j, i] * carbon_cost[i]
        PV_carbon_cost[j, i] = PV_AWE_LCA.iloc[j, i] * carbon_cost[i]
        Wind_carbon_cost[j, i] = Wind_AWE_LCA.iloc[j, i] * carbon_cost[i]
        Integrated_carbon_cost[j, i] = Integrated_AWE_LCA.iloc[j, i] * carbon_cost[i]

CG_carbon_cost_df = pd.DataFrame(CG_carbon_cost, index=Province, columns=Year)
SMR_carbon_cost_df = pd.DataFrame(SMR_carbon_cost, index=Province, columns=Year)
CG_CCS_carbon_cost_df = pd.DataFrame(CG_CCS_carbon_cost, index=Province, columns=Year)
SMR_CCS_carbon_cost_df = pd.DataFrame(SMR_CCS_carbon_cost, index=Province, columns=Year)
PV_carbon_cost_df = pd.DataFrame(PV_carbon_cost, index=Province, columns=Year)
Wind_carbon_cost_df = pd.DataFrame(Wind_carbon_cost, index=Province, columns=Year)
Integrated_carbon_cost_df = pd.DataFrame(
    Integrated_carbon_cost, index=Province, columns=Year
)


# 每五年线性平均至每年
CG_carbon_cost_18_60_df = calculate_lcoa_results(CG_carbon_cost_df)
CG_CCS_carbon_cost_18_60_df = calculate_lcoa_results(CG_CCS_carbon_cost_df)
SMR_carbon_cost_18_60_df = calculate_lcoa_results(SMR_carbon_cost_df)
SMR_CCS_carbon_cost_18_60_df = calculate_lcoa_results(SMR_CCS_carbon_cost_df)
PV_carbon_cost_18_60_df = calculate_lcoa_results(PV_carbon_cost_df)
Wind_carbon_cost_18_60_df = calculate_lcoa_results(Wind_carbon_cost_df)
Integrated_carbon_cost_18_60_df = calculate_lcoa_results(Integrated_carbon_cost_df)


LCOA_carbon_cost_18_60 = np.zeros((31, 301))

for i in range(43):
    LCOA_carbon_cost_18_60[:, 0 + i * 7] = CG_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 1 + i * 7] = SMR_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 2 + i * 7] = CG_CCS_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 3 + i * 7] = SMR_CCS_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 4 + i * 7] = PV_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 5 + i * 7] = Wind_carbon_cost_18_60_df.iloc[:, i]
    LCOA_carbon_cost_18_60[:, 6 + i * 7] = Integrated_carbon_cost_18_60_df.iloc[:, i]

LCOA_carbon_cost_18_60_df = pd.DataFrame(LCOA_carbon_cost_18_60, index=Province)

LCOA_all_18_60 = np.zeros((31, 301))
for i in range(301):
    for j in range(31):
        LCOA_all_18_60[j, i] = (
            LCOA_carbon_cost_18_60_df.iloc[j, i] + LCOA_18_60_df.iloc[j, i]
        )
LCOA_all_18_60_df = pd.DataFrame(LCOA_all_18_60, index=Province)
LCOA_all_18_60_df

d:\py\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\py\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
Anhui,3.724672,4.327076,5.161054,4.752156,8.250243,7.052651,5.650704,4.073745,4.513663,5.243016,...,4.995543,4.802963,4.703483,13.056520,7.990335,7.014597,5.762052,5.107926,4.840339,4.824893
Beijing,3.577310,4.135376,5.013872,4.560522,7.672479,9.071484,6.416756,3.925960,4.320774,5.095404,...,4.646750,6.174086,4.252227,12.953620,7.923415,6.912510,5.694908,4.751269,6.222074,4.363008
Chongqing,3.692834,3.376092,5.129026,3.801102,9.445482,8.952449,6.410485,4.039431,3.555729,5.208476,...,5.717102,6.093241,5.227516,13.025361,7.041255,6.983257,4.812898,5.845752,6.140604,5.366428
Fujian,3.709195,4.616607,6.987851,5.716610,7.411089,5.976575,4.769069,4.050749,4.782091,7.032467,...,4.488950,4.072128,3.952468,13.102581,8.452569,8.101402,6.604844,4.589911,4.103848,4.032531
Gansu,3.442416,3.501823,4.877989,3.926606,6.680390,6.472833,5.025077,3.784355,3.668388,4.952714,...,4.047831,4.409170,3.800822,12.781684,7.185908,6.739058,4.957279,4.138848,4.443499,3.898242
Guangdong,3.749437,5.013750,7.028205,6.113794,7.453406,8.119204,5.957844,4.093405,5.186009,7.075270,...,4.514496,5.527329,4.348562,13.087543,8.694573,8.085689,6.847250,4.616034,5.570312,4.458923
Guangxi,3.945035,4.830713,7.223889,5.930789,7.772177,7.098904,5.665244,4.286816,4.996834,7.268735,...,4.706937,4.834376,4.469072,13.287462,8.523663,8.285756,6.676343,4.812813,4.871996,4.583853
Guizhou,3.464748,4.386944,4.900653,4.811848,7.684801,8.038925,5.967833,3.807935,4.557010,4.976644,...,4.654188,5.472806,4.563779,12.811287,8.091432,6.769096,5.862878,4.758875,5.515367,4.680074
Hainan,3.663596,4.464924,5.100027,4.890022,6.628008,9.738079,5.997433,4.007534,4.637100,5.176780,...,4.016208,6.626815,3.849963,12.981084,8.087883,6.939006,5.859712,4.106512,6.678308,3.946020
Hebei,3.471977,4.149443,4.907938,4.574368,7.623586,6.681214,5.358591,3.823595,4.343172,4.992480,...,4.617233,4.550695,4.226037,12.782695,7.753403,6.740050,5.525103,4.721087,4.586119,4.336547


<IPython.core.display.Javascript object>

In [45]:
LCOA_df_national_all_18_60 = np.zeros((7, 43))
for i in range(43):
    for z in range(7):
        for j in range(7):
            LCOA_df_national_all_18_60[j, i] = LCOA_all_18_60_df.iloc[
                30, j + (z + 1) * i
            ]
LCOA_df_national_all_18_60_df = pd.DataFrame(
    np.transpose(LCOA_df_national_all_18_60),
    index=Year1,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_df_national_all_18_60_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
2018,3.570909,4.144863,5.191231,4.637329,7.579714,7.241205,5.607758
2019,3.917573,4.324686,5.267776,4.720678,7.373221,7.097941,5.521409
2020,4.264236,4.504509,5.344321,4.804027,7.166728,6.954676,5.435061
2021,4.456590,4.582519,5.362509,4.821135,6.937460,6.800335,5.326910
2022,4.648943,4.660529,5.380697,4.838244,6.708192,6.645995,5.218760
2023,4.841297,4.738540,5.398885,4.855353,6.478923,6.491654,5.110609
2024,5.033651,4.816550,5.417073,4.872461,6.249655,6.337313,5.002458
2025,5.226005,4.894560,5.435260,4.889570,6.020387,6.182972,4.894308
2026,5.277858,4.915209,5.410784,4.885394,5.782337,6.017365,4.761780
2027,5.329710,4.935857,5.386308,4.881218,5.544287,5.851757,4.629252


<IPython.core.display.Javascript object>

In [46]:
# 求出每年不同制氨技术成本的最小值 - 加碳价成本

LCOA_all_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_all_18_60_min[j, i] = np.min(
            LCOA_all_18_60_df.iloc[j, (7 * i) : (7 * i + 7)]
        )
LCOA_all_18_60_min_df = pd.DataFrame(LCOA_all_18_60_min)
# 各省2018-2060年制氨的最小潜力值
LCOA_all_18_60_min_sum = pd.DataFrame(
    (LCOA_all_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_all"],
)
LCOA_all_18_60_min_sum

,min_all
Anhui,4.273359
Beijing,3.994926
Chongqing,4.089004
Fujian,3.785654
Gansu,3.562834
Guangdong,4.126444
Guangxi,4.188648
Guizhou,4.225357
Hainan,3.745193
Hebei,3.935055


<IPython.core.display.Javascript object>

In [47]:
LCOA_all_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if LCOA_all_18_60_df.iloc[j, 7 * i + 0] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "A"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 1] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "B"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 2] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "C"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 3] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "D"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 4] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "E"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 5] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "F"
        elif LCOA_all_18_60_df.iloc[j, 7 * i + 6] == LCOA_all_18_60_min_df.iloc[j, i]:
            LCOA_all_18_60_min_type[j, i] = "G"
LCOA_all_18_60_min_type_df = pd.DataFrame(
    LCOA_all_18_60_min_type, index=Province, columns=Year1
)
LCOA_all_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,A,A,A,A,A,B,D,D,G,G,...,G,G,G,G,G,G,G,G,G,G
Beijing,A,A,A,A,A,B,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Chongqing,B,B,B,B,B,B,D,D,D,D,...,D,D,D,D,D,D,D,D,D,D
Fujian,A,A,A,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Gansu,A,B,B,B,B,B,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Guangdong,A,A,A,A,A,A,A,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guangxi,A,A,A,A,A,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guizhou,A,A,A,A,A,A,A,D,D,G,...,G,G,G,G,G,G,G,G,G,G
Hainan,A,A,A,A,A,A,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Hebei,A,A,A,A,A,A,D,G,G,G,...,G,G,G,G,G,G,G,G,G,G


<IPython.core.display.Javascript object>

In [48]:
LCOA_all_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_all_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "A"
    )
    LCOA_all_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "B"
    )
    LCOA_all_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "C"
    )
    LCOA_all_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "D"
    )
    LCOA_all_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "E"
    )
    LCOA_all_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "F"
    )
    LCOA_all_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_all_18_60_min_type[j, :] == "G"
    )
LCOA_all_18_60_min_type_province_df = pd.DataFrame(
    LCOA_all_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_all_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,5.0,1.0,0.0,2.0,0.0,0.0,35.0
Beijing,5.0,1.0,0.0,4.0,0.0,0.0,33.0
Chongqing,0.0,6.0,0.0,37.0,0.0,0.0,0.0
Fujian,3.0,0.0,0.0,0.0,0.0,0.0,40.0
Gansu,1.0,5.0,0.0,4.0,0.0,0.0,33.0
Guangdong,7.0,0.0,0.0,0.0,0.0,0.0,36.0
Guangxi,5.0,0.0,0.0,0.0,0.0,0.0,38.0
Guizhou,7.0,0.0,0.0,2.0,0.0,0.0,34.0
Hainan,6.0,0.0,0.0,0.0,0.0,0.0,37.0
Hebei,6.0,0.0,0.0,1.0,0.0,0.0,36.0


<IPython.core.display.Javascript object>

In [49]:
# 求出每年传统制氨技术成本的最小值 - 加碳成本

LCOA_all_conventional_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_all_conventional_18_60_min[j, i] = np.min(
            LCOA_all_18_60_df.iloc[j, (7 * i) : (7 * i + 2)]
        )
LCOA_all_conventional_18_60_min_df = pd.DataFrame(LCOA_all_conventional_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_all_conventional_18_60_min_sum = pd.DataFrame(
    (LCOA_all_conventional_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_all_conventional"],
)
LCOA_all_conventional_18_60_min_sum

,min_all_conventional
Anhui,5.500888
Beijing,5.400237
Chongqing,4.629787
Fujian,5.850561
Gansu,4.724751
Guangdong,6.044463
Guangxi,6.028336
Guizhou,5.590032
Hainan,5.594416
Hebei,5.299875


<IPython.core.display.Javascript object>

In [50]:
LCOA_all_conventional_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if (
            LCOA_all_18_60_df.iloc[j, 7 * i + 0]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "A"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 1]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "B"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 2]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "C"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 3]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "D"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 4]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "E"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 5]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "F"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 6]
            == LCOA_all_conventional_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_conventional_18_60_min_type[j, i] = "G"
LCOA_all_conventional_18_60_min_type_df = pd.DataFrame(
    LCOA_all_conventional_18_60_min_type, index=Province, columns=Year1
)
LCOA_all_conventional_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,A,A,A,A,A,B,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Beijing,A,A,A,A,A,B,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Chongqing,B,B,B,B,B,B,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Fujian,A,A,A,A,A,A,A,A,A,B,...,B,B,B,B,B,B,B,B,B,B
Gansu,A,B,B,B,B,B,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Guangdong,A,A,A,A,A,A,A,A,A,A,...,B,B,B,B,B,B,B,B,B,B
Guangxi,A,A,A,A,A,A,A,A,B,B,...,B,B,B,B,B,B,B,B,B,B
Guizhou,A,A,A,A,A,A,A,A,A,B,...,B,B,B,B,B,B,B,B,B,B
Hainan,A,A,A,A,A,A,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B
Hebei,A,A,A,A,A,A,B,B,B,B,...,B,B,B,B,B,B,B,B,B,B


<IPython.core.display.Javascript object>

In [51]:
LCOA_all_conventional_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_all_conventional_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "A"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "B"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "C"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "D"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "E"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "F"
    )
    LCOA_all_conventional_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_all_conventional_18_60_min_type[j, :] == "G"
    )
LCOA_all_conventional_18_60_min_type_province_df = pd.DataFrame(
    LCOA_all_conventional_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_all_conventional_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,5.0,38.0,0.0,0.0,0.0,0.0,0.0
Beijing,5.0,38.0,0.0,0.0,0.0,0.0,0.0
Chongqing,0.0,43.0,0.0,0.0,0.0,0.0,0.0
Fujian,9.0,34.0,0.0,0.0,0.0,0.0,0.0
Gansu,1.0,42.0,0.0,0.0,0.0,0.0,0.0
Guangdong,20.0,23.0,0.0,0.0,0.0,0.0,0.0
Guangxi,8.0,35.0,0.0,0.0,0.0,0.0,0.0
Guizhou,9.0,34.0,0.0,0.0,0.0,0.0,0.0
Hainan,6.0,37.0,0.0,0.0,0.0,0.0,0.0
Hebei,6.0,37.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [52]:
# 求出每年低碳制氨技术成本的最小值 - 加碳成本

LCOA_all_low_carbon_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_all_low_carbon_18_60_min[j, i] = np.min(
            LCOA_all_18_60_df.iloc[j, (7 * i+2) : (7 * i +7)]
        )
LCOA_all_low_carbon_18_60_min_df = pd.DataFrame(LCOA_all_low_carbon_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_all_low_carbon_18_60_min_sum = pd.DataFrame(
    (LCOA_all_low_carbon_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_all_low_carbon"],
)
LCOA_all_low_carbon_18_60_min_sum


,min_all_low_carbon
Anhui,4.339567
Beijing,4.056375
Chongqing,4.119993
Fujian,3.831797
Gansu,3.595182
Guangdong,4.305218
Guangxi,4.298513
Guizhou,4.337305
Hainan,3.836075
Hebei,4.011080


<IPython.core.display.Javascript object>

In [53]:
LCOA_all_low_carbon_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if (
            LCOA_all_18_60_df.iloc[j, 7 * i + 0]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "A"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 1]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "B"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 2]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "C"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 3]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "D"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 4]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "E"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 5]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "F"
        elif (
            LCOA_all_18_60_df.iloc[j, 7 * i + 6]
            == LCOA_all_low_carbon_18_60_min_df.iloc[j, i]
        ):
            LCOA_all_low_carbon_18_60_min_type[j, i] = "G"
            
LCOA_all_low_carbon_18_60_min_type_df = pd.DataFrame(
    LCOA_all_low_carbon_18_60_min_type, index=Province, columns=Year1
)
LCOA_all_low_carbon_18_60_min_type_df


,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,D,D,D,D,D,D,D,D,G,G,...,G,G,G,G,G,G,G,G,G,G
Beijing,D,D,D,D,D,D,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Chongqing,D,D,D,D,D,D,D,D,D,D,...,D,D,D,D,D,D,D,D,D,D
Fujian,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Gansu,D,D,D,D,D,D,D,D,D,D,...,G,G,G,G,G,G,G,G,G,G
Guangdong,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guangxi,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Guizhou,D,D,D,D,D,D,D,D,D,G,...,G,G,G,G,G,G,G,G,G,G
Hainan,D,D,D,D,D,D,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
Hebei,D,D,D,D,D,D,D,G,G,G,...,G,G,G,G,G,G,G,G,G,G


<IPython.core.display.Javascript object>

In [54]:
LCOA_all_low_carbon_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_all_low_carbon_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "A"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "B"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "C"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "D"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "E"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "F"
    )
    LCOA_all_low_carbon_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_all_low_carbon_18_60_min_type[j, :] == "G"
    )
LCOA_all_low_carbon_18_60_min_type_province_df = pd.DataFrame(
    LCOA_all_low_carbon_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind",'Integrated'],
)
LCOA_all_low_carbon_18_60_min_type_province_df


,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,0.0,0.0,0.0,8.0,0.0,0.0,35.0
Beijing,0.0,0.0,0.0,10.0,0.0,0.0,33.0
Chongqing,0.0,0.0,0.0,43.0,0.0,0.0,0.0
Fujian,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Gansu,0.0,0.0,0.0,10.0,0.0,0.0,33.0
Guangdong,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Guangxi,0.0,0.0,0.0,0.0,0.0,0.0,43.0
Guizhou,0.0,0.0,0.0,9.0,0.0,0.0,34.0
Hainan,0.0,0.0,0.0,6.0,0.0,0.0,37.0
Hebei,0.0,0.0,0.0,7.0,0.0,0.0,36.0


<IPython.core.display.Javascript object>

In [55]:
LCOA_all_two_type_18_60_min_sum = pd.concat(
    [LCOA_all_conventional_18_60_min_sum, LCOA_all_low_carbon_18_60_min_sum], axis=1,
)
LCOA_all_two_type_18_60_min_sum
LCOA_all_two_type_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\5 LCOA_all_two_type_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3916328442.py:5: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_all_two_type_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [56]:
# 求出每年传统制氨技术成本的最小值 - 不加碳成本

LCOA_blue_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_blue_18_60_min[j, i] = np.min(
            LCOA_18_60_df.iloc[j, (7 * i + 2) : (7 * i + 4)]
        )
LCOA_blue_18_60_min_df = pd.DataFrame(LCOA_blue_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_blue_18_60_min_sum = pd.DataFrame(
    (LCOA_blue_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_blue"],
)
LCOA_blue_18_60_min_sum

,min_blue
Anhui,4.472043
Beijing,4.307849
Chongqing,3.631149
Fujian,5.361062
Gansu,3.756654
Guangdong,5.685299
Guangxi,5.575241
Guizhou,4.284479
Hainan,4.466717
Hebei,4.243931


<IPython.core.display.Javascript object>

In [57]:
LCOA_blue_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\10 LCOA_blue_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1011977306.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_blue_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [58]:
# 求出每年传统制氨技术成本的最小值 - 不加碳成本

LCOA_green_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_green_18_60_min[j, i] = np.min(
            LCOA_18_60_df.iloc[j, (7 * i+4) : (7 * i + 7)]
        )
LCOA_green_18_60_min_df = pd.DataFrame(LCOA_green_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_green_18_60_min_sum = pd.DataFrame(
    (LCOA_green_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_green"],
)
LCOA_green_18_60_min_sum


,min_green
Anhui,3.902294
Beijing,3.757859
Chongqing,4.374172
Fujian,3.503972
Gansu,3.286810
Guangdong,3.803909
Guangxi,3.792825
Guizhou,3.936810
Hainan,3.468825
Hebei,3.583822


<IPython.core.display.Javascript object>

In [59]:
LCOA_green_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\11 LCOA_green_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3170296211.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_green_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [60]:
# 求出每年传统制氨技术成本的最小值 - 不加碳成本

LCOA_all_blue_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_all_blue_18_60_min[j, i] = np.min(
            LCOA_all_18_60_df.iloc[j, (7 * i + 2) : (7 * i + 4)]
        )
LCOA_all_blue_18_60_min_df = pd.DataFrame(LCOA_all_blue_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_all_blue_18_60_min_sum = pd.DataFrame(
    (LCOA_all_blue_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_blue_all"],
)
LCOA_all_blue_18_60_min_sum

,min_blue_all
Anhui,5.026474
Beijing,4.920923
Chongqing,4.119993
Fujian,5.911224
Gansu,4.216180
Guangdong,6.213575
Guangxi,6.084281
Guizhou,5.096172
Hainan,5.144733
Hebei,4.835180


<IPython.core.display.Javascript object>

In [61]:
LCOA_all_blue_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\12 LCOA_all_blue_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/3286532159.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_all_blue_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [62]:
# 求出每年传统制氨技术成本的最小值 - 不加碳成本

LCOA_all_green_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_all_green_18_60_min[j, i] = np.min(
            LCOA_all_18_60_df.iloc[j, (7 * i + 4) : (7 * i + 7)]
        )
LCOA_all_green_18_60_min_df = pd.DataFrame(LCOA_all_green_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_all_green_18_60_min_sum = pd.DataFrame(
    (LCOA_all_green_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_green_all"],
)
LCOA_all_green_18_60_min_sum

,min_green_all
Anhui,4.418540
Beijing,4.268685
Chongqing,4.981578
Fujian,3.831797
Gansu,3.725322
Guangdong,4.305218
Guangxi,4.298513
Guizhou,4.461644
Hainan,3.917071
Hebei,4.071375


<IPython.core.display.Javascript object>

In [63]:
LCOA_all_green_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\13 LCOA_all_green_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1557174635.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_all_green_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [64]:
LCA_df = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\2 LCA\LCA.xlsx", "LCA",
)

CG_LCA_ele = LCA_df.iloc[35:66, 1:11]
SMR_LCA_ele = LCA_df.iloc[105:136, 1:11]
CG_CCS_LCA_ele = LCA_df.iloc[70:101, 1:11]
SMR_CCS_LCA_ele = LCA_df.iloc[140:171, 1:11]
PV_AWE_LCA_ele = pd.DataFrame(np.zeros((31, 10)))
Wind_AWE_LCA_ele = pd.DataFrame(np.zeros((31, 10)))
Integrated_AWE_LCA_ele = pd.DataFrame(np.zeros((31, 10)))

CG_carbon_cost_ele = np.zeros((31, 10))
SMR_carbon_cost_ele = np.zeros((31, 10))
CG_CCS_carbon_cost_ele = np.zeros((31, 10))
SMR_CCS_carbon_cost_ele = np.zeros((31, 10))
PV_carbon_cost_ele = np.zeros((31, 10))
Wind_carbon_cost_ele = np.zeros((31, 10))
Integrated_carbon_cost_ele = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        CG_carbon_cost_ele[j, i] = CG_LCA_ele.iloc[j, i] * carbon_cost[i]
        SMR_carbon_cost_ele[j, i] = SMR_LCA_ele.iloc[j, i] * carbon_cost[i]
        CG_CCS_carbon_cost_ele[j, i] = CG_CCS_LCA_ele.iloc[j, i] * carbon_cost[i]
        SMR_CCS_carbon_cost_ele[j, i] = SMR_CCS_LCA_ele.iloc[j, i] * carbon_cost[i]
        PV_carbon_cost_ele[j, i] = PV_AWE_LCA_ele.iloc[j, i] * carbon_cost[i]
        Wind_carbon_cost_ele[j, i] = Wind_AWE_LCA_ele.iloc[j, i] * carbon_cost[i]
        Integrated_carbon_cost_ele[j, i] = (
            Integrated_AWE_LCA_ele.iloc[j, i] * carbon_cost[i]
        )

CG_carbon_cost_ele_df = pd.DataFrame(CG_carbon_cost_ele, index=Province, columns=Year)
SMR_carbon_cost_ele_df = pd.DataFrame(SMR_carbon_cost_ele, index=Province, columns=Year)
CG_CCS_carbon_cost_ele_df = pd.DataFrame(
    CG_CCS_carbon_cost_ele, index=Province, columns=Year
)
SMR_CCS_carbon_cost_ele_df = pd.DataFrame(
    SMR_CCS_carbon_cost_ele, index=Province, columns=Year
)
PV_carbon_cost_ele_df = pd.DataFrame(PV_carbon_cost_ele, index=Province, columns=Year)
Wind_carbon_cost_ele_df = pd.DataFrame(
    Wind_carbon_cost_ele, index=Province, columns=Year
)
Integrated_carbon_cost_ele_df = pd.DataFrame(
    Integrated_carbon_cost_ele, index=Province, columns=Year
)


# 每五年线性平均至每年
CG_carbon_cost_ele_18_60_df = calculate_lcoa_results(CG_carbon_cost_ele_df)
CG_CCS_carbon_cost_ele_18_60_df = calculate_lcoa_results(CG_CCS_carbon_cost_ele_df)
SMR_carbon_cost_ele_18_60_df = calculate_lcoa_results(SMR_carbon_cost_ele_df)
SMR_CCS_carbon_cost_ele_18_60_df = calculate_lcoa_results(SMR_CCS_carbon_cost_ele_df)
PV_carbon_cost_ele_18_60_df = calculate_lcoa_results(PV_carbon_cost_ele_df)
Wind_carbon_cost_ele_18_60_df = calculate_lcoa_results(Wind_carbon_cost_ele_df)
Integrated_carbon_cost_ele_18_60_df = calculate_lcoa_results(
    Integrated_carbon_cost_ele_df
)

LCOA_carbon_cost_ele_18_60 = np.zeros((31, 301))

for i in range(43):
    LCOA_carbon_cost_ele_18_60[:, 0 + i * 7] = CG_carbon_cost_ele_18_60_df.iloc[:, i]
    LCOA_carbon_cost_ele_18_60[:, 1 + i * 7] = SMR_carbon_cost_ele_18_60_df.iloc[:, i]
    LCOA_carbon_cost_ele_18_60[:, 2 + i * 7] = CG_CCS_carbon_cost_ele_18_60_df.iloc[
        :, i
    ]
    LCOA_carbon_cost_ele_18_60[:, 3 + i * 7] = SMR_CCS_carbon_cost_ele_18_60_df.iloc[
        :, i
    ]
    LCOA_carbon_cost_ele_18_60[:, 4 + i * 7] = PV_carbon_cost_ele_18_60_df.iloc[:, i]
    LCOA_carbon_cost_ele_18_60[:, 5 + i * 7] = Wind_carbon_cost_ele_18_60_df.iloc[:, i]
    LCOA_carbon_cost_ele_18_60[:, 6 + i * 7] = Integrated_carbon_cost_ele_18_60_df.iloc[
        :, i
    ]

LCOA_carbon_cost_ele_18_60_df = pd.DataFrame(LCOA_carbon_cost_ele_18_60, index=Province)

LCOA_ele_18_60 = np.zeros((31, 301))
for i in range(301):
    for j in range(31):
        LCOA_ele_18_60[j, i] = (
            LCOA_carbon_cost_ele_18_60_df.iloc[j, i] + LCOA_18_60_df.iloc[j, i]
        )
LCOA_ele_18_60_df = pd.DataFrame(LCOA_ele_18_60, index=Province)
LCOA_ele_18_60_df

d:\py\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\py\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
Anhui,3.724672,4.327076,5.161054,4.752156,8.250243,7.052651,5.650704,3.746632,4.388706,5.146233,...,3.442648,4.129471,3.072416,3.765300,4.441098,4.265590,4.624033,3.422158,4.109221,3.054433
Beijing,3.577310,4.135376,5.013872,4.560522,7.672479,9.071484,6.416756,3.598846,4.195817,4.998621,...,3.202576,5.308583,2.767917,3.662400,4.374178,4.163503,4.556889,3.183525,5.282515,2.751764
Chongqing,3.692834,3.376092,5.129026,3.801102,9.445482,8.952449,6.410485,3.712318,3.430772,5.111692,...,3.939293,5.239060,3.373634,3.734142,3.492018,4.234250,3.674879,3.915826,5.213335,3.354024
Fujian,3.709195,4.616607,6.987851,5.716610,7.411089,5.976575,4.769069,3.723636,4.657134,6.935684,...,3.093963,3.500982,2.904991,3.811362,4.903332,5.352395,5.466825,3.075564,3.483833,2.889966
Gansu,3.442416,3.501823,4.877989,3.926606,6.680390,6.472833,5.025077,3.457242,3.543431,4.855930,...,2.790344,3.790825,2.491144,3.490465,3.636671,3.990051,3.819260,2.773764,3.772245,2.476561
Guangdong,3.749437,5.013750,7.028205,6.113794,7.453406,8.119204,5.957844,3.766292,5.061052,6.978486,...,3.111547,4.752398,2.861256,3.796324,5.145336,5.336682,5.709230,3.093042,4.729075,2.844440
Guangxi,3.945035,4.830713,7.223889,5.930789,7.772177,7.098904,5.665244,3.959702,4.871877,7.171952,...,3.244002,4.156486,2.924802,3.996243,4.974427,5.536749,5.538324,3.224703,4.136102,2.907671
Guizhou,3.464748,4.386944,4.900653,4.811848,7.684801,8.038925,5.967833,3.480821,4.432053,4.879861,...,3.207696,4.705510,2.994969,3.520067,4.542195,4.020089,4.724859,3.188614,4.682418,2.977395
Hainan,3.663596,4.464924,5.100027,4.890022,6.628008,9.738079,5.997433,3.680421,4.512143,5.079997,...,2.768578,5.697912,2.548674,3.689865,4.538647,4.189999,4.721693,2.752129,5.669924,2.533649
Hebei,3.471977,4.149443,4.907938,4.574368,7.623586,6.681214,5.358591,3.496482,4.218215,4.895697,...,3.182260,3.912531,2.746634,3.491476,4.204166,3.991043,4.387083,3.163331,3.893351,2.730625


<IPython.core.display.Javascript object>

In [65]:
LCOA_df_national_ele_18_60 = np.zeros((7, 43))
for i in range(43):
    for z in range(7):
        for j in range(7):
            LCOA_df_national_ele_18_60[j, i] = LCOA_ele_18_60_df.iloc[
                30, j + (z + 1) * i
            ]
LCOA_df_national_ele_18_60_df = pd.DataFrame(
    np.transpose(LCOA_df_national_ele_18_60),
    index=Year1,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_df_national_ele_18_60_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
2018,3.570909,4.144863,5.191231,4.637329,7.579714,7.241205,5.607758
2019,3.590459,4.199729,5.170993,4.680612,7.318693,7.071421,5.484301
2020,3.610009,4.254595,5.150754,4.723895,7.057672,6.901638,5.360844
2021,3.611878,4.259840,5.112583,4.717672,6.796651,6.731855,5.227791
2022,3.613747,4.265085,5.074412,4.711450,6.535630,6.562071,5.094738
2023,3.615616,4.270331,5.036241,4.705227,6.274609,6.392288,4.961685
2024,3.617485,4.275576,4.998070,4.699005,6.013588,6.222504,4.828632
2025,3.619354,4.280821,4.959898,4.692782,5.752567,6.052721,4.695579
2026,3.619701,4.281795,4.920183,4.682297,5.505932,5.882938,4.549538
2027,3.620048,4.282768,4.880468,4.671812,5.259296,5.713154,4.403497


<IPython.core.display.Javascript object>

In [66]:
LCOA_df_national_ele_18_60_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\14 LCOA_df_national_ele_18_60_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1239012207.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_df_national_ele_18_60_df.to_excel(


<IPython.core.display.Javascript object>

In [67]:
# 求出每年不同制氨技术成本的最小值 - 加碳价成本ele

LCOA_ele_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_ele_18_60_min[j, i] = np.min(
            LCOA_ele_18_60_df.iloc[j, (7 * i) : (7 * i + 7)]
        )
LCOA_ele_18_60_min_df = pd.DataFrame(LCOA_ele_18_60_min)
# 各省2018-2060年制氨的最小潜力值
LCOA_ele_18_60_min_sum = pd.DataFrame(
    (LCOA_ele_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_ele"],
)

<IPython.core.display.Javascript object>

In [68]:
LCOA_ele_18_60_min_type = np.empty((31, 43), dtype="str")

for i in range(43):
    for j in range(31):
        if LCOA_ele_18_60_df.iloc[j, 7 * i + 0] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "A"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 1] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "B"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 2] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "C"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 3] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "D"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 4] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "E"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 5] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "F"
        elif LCOA_ele_18_60_df.iloc[j, 7 * i + 6] == LCOA_ele_18_60_min_df.iloc[j, i]:
            LCOA_ele_18_60_min_type[j, i] = "G"
LCOA_ele_18_60_min_type_df = pd.DataFrame(
    LCOA_ele_18_60_min_type, index=Province, columns=Year1
)
LCOA_ele_18_60_min_type_df

,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
Anhui,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Beijing,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Chongqing,B,B,B,B,B,B,B,B,B,B,...,B,B,B,B,B,G,G,G,G,G
Fujian,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Gansu,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guangdong,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guangxi,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Guizhou,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Hainan,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
Hebei,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G


<IPython.core.display.Javascript object>

In [69]:
LCOA_ele_18_60_min_type_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\15 LCOA_ele_18_60_min_type_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2727265870.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_18_60_min_type_df.to_excel(


<IPython.core.display.Javascript object>

In [70]:
LCOA_ele_18_60_min_type_province = np.zeros((31, 7))
for j in range(31):
    LCOA_ele_18_60_min_type_province[j, 0] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "A"
    )
    LCOA_ele_18_60_min_type_province[j, 1] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "B"
    )
    LCOA_ele_18_60_min_type_province[j, 2] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "C"
    )
    LCOA_ele_18_60_min_type_province[j, 3] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "D"
    )
    LCOA_ele_18_60_min_type_province[j, 4] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "E"
    )
    LCOA_ele_18_60_min_type_province[j, 5] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "F"
    )
    LCOA_ele_18_60_min_type_province[j, 6] = np.count_nonzero(
        LCOA_ele_18_60_min_type[j, :] == "G"
    )
LCOA_ele_18_60_min_type_province_df = pd.DataFrame(
    LCOA_ele_18_60_min_type_province,
    index=Province,
    columns=["CG", "SMR", "CG+CCS", "SMR+CCS", "PV", "Wind", "Integrated"],
)
LCOA_ele_18_60_min_type_province_df

,CG,SMR,CG+CCS,SMR+CCS,PV,Wind,Integrated
Anhui,20.0,0.0,0.0,0.0,0.0,0.0,23.0
Beijing,17.0,0.0,0.0,0.0,0.0,0.0,26.0
Chongqing,0.0,38.0,0.0,0.0,0.0,0.0,5.0
Fujian,12.0,0.0,0.0,0.0,0.0,0.0,31.0
Gansu,14.0,0.0,0.0,0.0,0.0,0.0,29.0
Guangdong,16.0,0.0,0.0,0.0,0.0,0.0,27.0
Guangxi,14.0,0.0,0.0,0.0,0.0,0.0,29.0
Guizhou,22.0,0.0,0.0,0.0,0.0,0.0,21.0
Hainan,12.0,0.0,0.0,0.0,0.0,0.0,31.0
Hebei,18.0,0.0,0.0,0.0,0.0,0.0,25.0


<IPython.core.display.Javascript object>

In [71]:
LCOA_ele_18_60_min_type_province_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\16 LCOA_ele_18_60_min_type_province_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2321003821.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_18_60_min_type_province_df.to_excel(


<IPython.core.display.Javascript object>

In [72]:
# 求出每年传统制氨技术成本的最小值 - 加碳成本

LCOA_ele_conventional_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_ele_conventional_18_60_min[j, i] = np.min(
            LCOA_ele_18_60_df.iloc[j, (7 * i) : (7 * i + 2)]
        )
LCOA_ele_conventional_18_60_min_df = pd.DataFrame(LCOA_ele_conventional_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_ele_conventional_18_60_min_sum = pd.DataFrame(
    (LCOA_ele_conventional_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_ele_conventional"],
)
LCOA_ele_conventional_18_60_min_sum

,min_ele_conventional
Anhui,3.754467
Beijing,3.637850
Chongqing,3.504388
Fujian,3.776810
Gansu,3.477660
Guangdong,3.783181
Guangxi,3.997963
Guizhou,3.521807
Hainan,3.686388
Hebei,3.496948


<IPython.core.display.Javascript object>

In [73]:
LCOA_ele_conventional_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\19 LCOA_ele_conventional_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/1352270121.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_conventional_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [74]:
LCOA_ele_blue_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_ele_blue_18_60_min[j, i] = np.min(
            LCOA_ele_18_60_df.iloc[j, (7 * i + 2) : (7 * i + 4)]
        )
LCOA_ele_blue_18_60_min_df = pd.DataFrame(LCOA_ele_blue_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_ele_blue_18_60_min_sum = pd.DataFrame(
    (LCOA_ele_blue_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_blue_ele"],
)
LCOA_ele_blue_18_60_min_sum

,min_blue_ele
Anhui,4.523218
Beijing,4.401160
Chongqing,3.759148
Fujian,5.544244
Gansu,3.855335
Guangdong,5.753545
Guangxi,5.723400
Guizhou,4.344040
Hainan,4.498242
Hebei,4.289140


<IPython.core.display.Javascript object>

In [75]:
LCOA_ele_blue_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\17 LCOA_ele_blue_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2799267294.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_blue_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [76]:
LCOA_ele_green_18_60_min = np.zeros((31, 43))
for i in range(43):
    for j in range(31):
        LCOA_ele_green_18_60_min[j, i] = np.min(
            LCOA_ele_18_60_df.iloc[j, (7 * i + 4) : (7 * i + 7)]
        )
LCOA_ele_green_18_60_min_df = pd.DataFrame(LCOA_ele_green_18_60_min)

# 各省2018-2060年制氨的最小潜力值
LCOA_ele_green_18_60_min_sum = pd.DataFrame(
    (LCOA_ele_green_18_60_min_df.values.sum(axis=1) / 43),
    index=Province,
    columns=["min_green_ele"],
)
LCOA_ele_green_18_60_min_sum

,min_green_ele
Anhui,3.902294
Beijing,3.757859
Chongqing,4.374172
Fujian,3.503972
Gansu,3.286810
Guangdong,3.803909
Guangxi,3.792825
Guizhou,3.936810
Hainan,3.468825
Hebei,3.583822


<IPython.core.display.Javascript object>

In [77]:
LCOA_ele_green_18_60_min_sum.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\18 LCOA_ele_green_18_60_min_sum.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2457084943.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_green_18_60_min_sum.to_excel(


<IPython.core.display.Javascript object>

In [78]:
LCOA_all_18_60_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\23 LCOA_all_18_60_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/2188618822.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_all_18_60_df.to_excel(


<IPython.core.display.Javascript object>

In [79]:
LCOA_ele_18_60_df.to_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\2 Python\4 code\3 LCC\Outputs\22 LCOA_ele_18_60_df.xlsx"
)

C:\Users\summer\AppData\Local\Temp/ipykernel_14168/4031124005.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  LCOA_ele_18_60_df.to_excel(


<IPython.core.display.Javascript object>